In [ ]:
#You will need to download the SEC Filings archive Zip Files
#Below Cleans and Joinging the 3 main SEC Datasets on company filings

import zipfile
import pandas as pd
import os
import time
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
zip_dir = "/content/drive/MyDrive/FSE IPO Data/Raw Zip Files/FSE570"
output_dir = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final"
os.makedirs(output_dir, exist_ok=True)
#output_file = os.path.join(output_dir, "SEC_filings_cleaned_final10.csv")

# Clean start
if os.path.exists(output_file):
    os.remove(output_file)
    print(f"🗑️ Deleted existing {output_file} for a clean rebuild.")

# Get sorted list of zip files
zip_files = sorted([f for f in os.listdir(zip_dir) if f.endswith(".zip")])
first_file = True  # Controls header writing

# Loop through each ZIP
for zip_filename in zip_files:
    zip_path = os.path.join(zip_dir, zip_filename)
    print(f"\n🔄 Processing: {zip_filename}")

    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            text_files = [f for f in zip_ref.namelist() if f.endswith('.txt')]
            temp_dfs = {}

            # Extract .txt files
            for text_file in text_files:
                with zip_ref.open(text_file) as file:
                    try:
                        content = file.read().decode('utf-8')
                    except UnicodeDecodeError:
                        content = file.read().decode('latin-1')

                    lines = content.strip().split('\n')
                    if not lines:
                        continue

                    header = lines[0].split('\t')
                    if len(header) == 1:
                        header = lines[0].split()

                    data = []
                    for line in lines[1:]:
                        values = line.split('\t')
                        if len(values) == 1:
                            values = line.split()
                        if len(values) == len(header):
                            data.append(dict(zip(header, values)))

                    filename = os.path.splitext(os.path.basename(text_file))[0]
                    temp_dfs[f"{filename}_df"] = pd.DataFrame(data)

            # Grab required DataFrames
            num_df = temp_dfs.get("num_df")
            tag_df = temp_dfs.get("tag_df")
            sub_df = temp_dfs.get("sub_df")

            if num_df is None or tag_df is None or sub_df is None:
                print(f"⚠️ Skipping {zip_filename}: missing num, tag, or sub.")
                continue

            if "custom" not in tag_df.columns or "abstract" not in tag_df.columns:
                print(f"⚠️ Skipping {zip_filename}: 'custom' or 'abstract' column missing.")
                continue

            # Filter tag table
            tag_df["custom"] = pd.to_numeric(tag_df["custom"], errors="coerce")
            tag_df["abstract"] = pd.to_numeric(tag_df["abstract"], errors="coerce")
            tag_filtered_df = tag_df[(tag_df["custom"] == 0) & (tag_df["abstract"] == 0)]
            print(f"   ✅ tag_filtered_df: {len(tag_filtered_df)} rows")

            # Join num + tag
            num_tag_df = pd.merge(num_df, tag_filtered_df, on=["tag", "version"], how="inner")
            print(f"   ✅ num_tag_df: {len(num_tag_df)} rows")

            # Join with sub
            combined_df = pd.merge(num_tag_df, sub_df, on="adsh", how="inner")
            print(f"   ✅ combined_df: {len(combined_df)} rows")

            # Add quarter info
            quarter = zip_filename.replace("-archive.zip", "")
            combined_df["filing_quarter"] = quarter

            if len(combined_df) == 0:
                print(f"   ⚠️ No rows to write for {zip_filename}. Skipping.")
                continue

            # Write to file
            combined_df.to_csv(output_file, mode='a', header=first_file, index=False)
            print(f"   💾 Appended {len(combined_df)} rows to {output_file}")
            first_file = False

            # Add sleep delay
            print("⏳ Waiting 15 seconds before next quarter...")
            time.sleep(15)

    except Exception as e:
        print(f"❌ Error processing {zip_filename}: {e}")

# Wrap up
print(f"\n🎉 All zip files processed.\n📁 Final data saved to:\n{output_file}")


In [ ]:
#Get Summary info on new Filings data
sample_df = pd.read_csv(output_file, nrows=100)


print("\n📋 Column Names:")
print(sample_df.columns.tolist())

print("\n🧪 Data Types:")
print(sample_df.dtypes)


summary = sample_df.describe(include='all').T
summary['example_value'] = sample_df.iloc[0]
summary = summary[['example_value']]
summary.reset_index(inplace=True)
summary.columns = ['column_name', 'example_value']

print("\n🔍 Sample Values from First Row:")
display(summary)

In [ ]:
#Second we Get Ticker Data from Yahoo Finance on Tickers price start, 6 month, and 3 years
 #YF is a free API but we found we were not able to sucessfuly get enough tickers for a proper test set so we purchased a Cheap API with EODHD, this code will be next cell block

#Getting Ticker Price Data from Yahoo Finance

import yfinance as yf
import pandas as pd
import datetime
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed


# Step 1: Load the full ticker list from the joined file.

input_path = "/content/my_drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_union_unique_joined.csv"
ticker_data = pd.read_csv(input_path, dtype={'ticker': str, 'cik': str})
ticker_data['ticker'] = ticker_data['ticker'].str.strip()
ticker_data = ticker_data.drop_duplicates(subset=["ticker", "cik"])
ticker_list = ticker_data.to_dict(orient="records")
print(f"🚀 Loaded {len(ticker_list):,} tickers for processing.")

# =============================================================================
# Step 2: Helper Function - Get the Price Nearest to a Target Date
# =============================================================================
def get_price_nearest_to(df, target_date):
    if df.empty or pd.isna(target_date):
        return pd.NA
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    nearest_date = min(df.index, key=lambda d: abs(d - target_date))
    price = df.loc[nearest_date]['Close']
    if isinstance(price, pd.Series):
        price = price.iloc[0]
    return price

# =============================================================================
# Step 3: Main Function to Fetch Ticker Data with Adjusted Prices
# =============================================================================
def get_ticker_info(row, max_retries=3):
    tkr = row['ticker']
    cik = row.get('cik', None)
    method_used = "failed"
    data = None

    # Define download methods (all using adjusted data)
    methods = [
        # Method 0: Try getting max available history.
        lambda: yf.download(tkr, period="max", auto_adjust=True, progress=False),
        # Method 1: Explicitly request from a very early date (1900) to today.
        lambda: yf.download(tkr, start="1900-01-01", end=datetime.date.today().strftime("%Y-%m-%d"), auto_adjust=True, progress=False),
        # Method 2: As a fallback, try a 10-year period.
        lambda: yf.download(tkr, period="10y", auto_adjust=True, progress=False)
    ]

    for i, method in enumerate(methods):
        retries = 0
        while retries < max_retries:
            try:
                data_try = method()
                if not data_try.empty:
                    data = data_try
                    # Label the method used for later reference.
                    method_used = ["max", "1900-now", "10y"][i]
                    break  # Exit the retry loop if data is successfully fetched.
            except Exception as e:
                print(f"Method {i} for ticker {tkr} failed on retry {retries+1} with error: {e}")
            retries += 1
            sleep_time = 2 ** retries  # Exponential backoff.
            print(f"Retrying ticker {tkr} in {sleep_time} seconds...")
            time.sleep(sleep_time)
        if data is not None and not data.empty:
            break  # Exit if we have successfully fetched data.

    # Add a small random delay to be a good API citizen.
    time.sleep(random.uniform(0.5, 1.5))

    if data is not None and not data.empty:
        if not isinstance(data.index, pd.DatetimeIndex):
            data.index = pd.to_datetime(data.index)
        start_date = data.index.min()
        end_date = data.index.max()
        six_months = start_date + pd.DateOffset(months=6)
        three_years = start_date + pd.DateOffset(years=3)
        price_start = get_price_nearest_to(data, start_date)
        price_6mo = get_price_nearest_to(data, six_months)
        price_3yr = get_price_nearest_to(data, three_years)
        return {
            "ticker": tkr,
            "cik": cik,
            "start_date": start_date,
            "end_date": end_date,
            "method": method_used,
            "date_6mo": six_months,
            "date_3yr": three_years,
            "price_start": price_start,
            "price_6mo": price_6mo,
            "price_3yr": price_3yr
        }
    else:
        print(f"❌ Failed to retrieve data for ticker {tkr} after {max_retries} retries per method.")
        return {
            "ticker": tkr,
            "cik": cik,
            "start_date": pd.NaT,
            "end_date": pd.NaT,
            "method": method_used,
            "date_6mo": pd.NaT,
            "date_3yr": pd.NaT,
            "price_start": pd.NA,
            "price_6mo": pd.NA,
            "price_3yr": pd.NA
        }

# =============================================================================
# Step 4: Process All Tickers Concurrently
# =============================================================================
max_threads = 5  # Adjust as needed based on your system and API limits.
results = []
start_time = time.time()
print(f"⚙️ Fetching data for {len(ticker_list):,} tickers using {max_threads} threads...")

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = [executor.submit(get_ticker_info, row) for row in ticker_list]
    for i, future in enumerate(as_completed(futures)):
        try:
            res = future.result()
            results.append(res)
            if (i+1) % 100 == 0:
                print(f"Processed {i+1} tickers so far...")
        except Exception as e:
            print(f"❌ Error processing ticker at index {i}: {e}")

end_time = time.time()
print(f"✅ Full run complete in {end_time - start_time:.2f} seconds!")

# =============================================================================
# Step 5: Create a DataFrame from the Results and Review Summary
# =============================================================================
df_results = pd.DataFrame(results)
print("=== Final Ticker Information ===")
print(df_results.head())

# Optional: Save the results to a CSV file if desired.
# Uncomment the following lines to save the full results back to Google Drive:
output_path = "/content/my_drive/MyDrive/FSE IPO Data/Filtered Data/Final/full_ticker_data_with_adjusted_prices.csv"
df_results.to_csv(output_path, index=False)
print("✅ Output saved to:", output_path)

In [ ]:
#Paid API Pull from EODHD financial repo

import pandas as pd
import requests
from datetime import datetime, timedelta
import logging

TICKERS = tickers

 #[t + ".US" for t in tickers[:5]]

# EODHD API key
API_KEY = "API KEY"

# --- Logging setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def fetch_ticker_data(ticker, from_dt, to_dt, api_key):
    url = (
        f"https://eodhd.com/api/eod/{ticker}"
        f"?from={from_dt}&to={to_dt}"
        f"&period=d&api_token={api_key}&fmt=json"
    )
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()
        if isinstance(data, dict) and data.get("warning"):
            logger.warning(f"{ticker}: {data['warning']}")
            return None
        if not data or 'date' not in data[0]:
            logger.warning(f"{ticker}: no usable data")
            return None

        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        return df[['adjusted_close']]
    except Exception as e:
        logger.error(f"{ticker}: fetch failed: {e}")
        return None

def calculate_metrics(ticker, start_str, df):
    # ensure sorted
    df = df.sort_index()

    # parse start date (must be in index)
    start_dt = pd.to_datetime(start_str)

    # exact first price
    try:
        sp = df.loc[start_dt, 'adjusted_close']
    except KeyError:
        # fallback if for some reason start_str missing
        sp = df['adjusted_close'].iloc[0]

    # six‐month and three‐year asof lookups
    six_m_dt = start_dt + timedelta(days=180)
    three_y_dt = start_dt + timedelta(days=365*3)

    s6 = df['adjusted_close'].asof(six_m_dt)
    s3 = df['adjusted_close'].asof(three_y_dt)

    # 30-day window around six-month date
    window = pd.date_range(six_m_dt - timedelta(days=15),
                           six_m_dt + timedelta(days=15))
    arr6 = df['adjusted_close'].loc[
        df.index.to_series().between(window.min(), window.max())
    ]
    avg6 = arr6.mean() if not arr6.empty else (s6 if pd.notna(s6) else None)

    # 31-day centered SMA and then average over 3-year period
    df['sma_30'] = df['adjusted_close'].rolling(31, center=True).mean()
    avg3 = df['sma_30'].loc[start_dt:three_y_dt].mean()

    return {
        'ticker':            ticker,
        'start_date':        start_str,
        'six_month_date':    six_m_dt.strftime('%Y-%m-%d'),
        'three_year_date':   three_y_dt.strftime('%Y-%m-%d'),
        'start_price':       round(sp, 2) if pd.notna(sp) else None,
        'six_month_price':   round(s6, 2) if pd.notna(s6) else None,
        'three_year_price':  round(s3, 2) if pd.notna(s3) else None,
        'six_month_30d_avg': round(avg6, 2) if pd.notna(avg6) else None,
        'three_year_30d_avg':round(avg3, 2) if pd.notna(avg3) else None,
    }

def test_batch_tickers():
    today = datetime.today().strftime('%Y-%m-%d')

    # prepare results DataFrame with all nine columns
    results_df = pd.DataFrame(columns=[
        'ticker','start_date','six_month_date','three_year_date',
        'start_price','six_month_price','three_year_price',
        'six_month_30d_avg','three_year_30d_avg'
    ])

    for ticker in TICKERS:
        logger.info(f"Fetching {ticker} …")
        df = fetch_ticker_data(ticker, "2000-01-01", today, API_KEY)
        if df is None or df.empty:
            logger.warning(f"Skipping {ticker}: no data")
            continue

        # actual first trading date from the fetched data
        actual_start = df.index.min()
        start_str = actual_start.strftime('%Y-%m-%d')

        # calculate and append
        metrics = calculate_metrics(ticker, start_str, df)
        results_df.loc[len(results_df)] = metrics

    print("----- Final Results DataFrame -----")
    display(results_df)

if __name__ == "__main__":
    test_batch_tickers()

file_path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_output_EODHD.csv'



In [ ]:
#third we get a mapping file created by joining two data sources from the SEC linking CIK (Centeral Index Key) to IPO Tickers. Look up SEC Company_tickers.josn and SEC ticker.txt

json_path = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/company_tickers.json"
with open(json_path, 'r') as f:
    data = json.load(f)

# Convert the JSON dictionary to a DataFrame.
cik_map_df = pd.DataFrame.from_dict(data, orient='index')
cik_map_df.columns = ['cik_str', 'ticker', 'title']

# Create a properly formatted 'cik' column padded to 10 characters.
cik_map_df['cik'] = cik_map_df['cik_str'].apply(lambda x: str(x).zfill(10))

# Rearrange columns (optionally drop the original 'cik_str').
cik_map_df = cik_map_df[['cik', 'ticker', 'title']]

print("✅ First few rows from the JSON file:")
print(cik_map_df.head())
print(f"Total records in JSON: {len(cik_map_df)}\n")

# --------------------------
# 2. Load and Process the Ticker Text File
# --------------------------
ticker_file_path = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker.txt"
ticker_df = pd.read_csv(ticker_file_path, sep='\t', names=["ticker", "cik"])

# Ensure the 'cik' values are 10 characters long (padded with zeros if needed).
ticker_df['cik'] = ticker_df['cik'].apply(lambda x: str(x).zfill(10))

# Convert all ticker values to uppercase to match the JSON file's format.
ticker_df['ticker'] = ticker_df['ticker'].str.upper()

print("✅ First few rows from the ticker text file (tickers capitalized):")
print(ticker_df.head())
print(f"Total records in ticker file: {len(ticker_df)}\n")

# --------------------------
# 3. Merge the DataFrames with an Outer Join
# --------------------------
merged_df = pd.merge(ticker_df, cik_map_df, on=['cik', 'ticker'], how='outer', indicator=True)

print("✅ Merged DataFrame preview:")
print(merged_df.head())
print("\nMerge source counts:")
print(merged_df['_merge'].value_counts())
print("\n")

# --------------------------
# 4. Update the Merge Indicator Labels
# --------------------------
# Replace the merge indicator values:
# - 'left_only' becomes 'ticker_txt'
# - 'right_only' becomes 'comp_tick_json'
# - 'both' stays unchanged
merged_df['_merge'] = merged_df['_merge'].replace({
    'left_only': 'ticker_txt',
    'right_only': 'comp_tick_json'
})

# --------------------------
# 5. Create the Union (ticker_union_unique)
# --------------------------
ticker_union_unique = merged_df.copy()

# --------------------------
# 6. Display the Union Results
# --------------------------
print("✅ Ticker Union Unique DataFrame (all unique rows with origin labels):")
print(ticker_union_unique.head())
print("\nFull merge source counts after label updates:")
print(ticker_union_unique['_merge'].value_counts())

# --------------------------
# 7. Save the Union Results to a CSV File
# --------------------------
ticker_union_unique.to_csv("/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_union_unique.csv", index=False)


In [ ]:

#finding number of unique companies (CIK) as some have multiple tickers

import pandas as pd
import os
import pandas as pd
from google.colab import drive

ipo_cik_file = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_union_unique.csv"
output_cik_file = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/cik_unique.csv"

# ✅ Load and extract unique CIKs
ipo_df = pd.read_csv(ipo_cik_file)

# Ensure CIK column exists and is normalized
ipo_df['cik'] = ipo_df['cik'].astype(str).str.zfill(10)
cik_unique_df = ipo_df[['cik']].drop_duplicates().sort_values('cik')

# ✅ Save to file
cik_unique_df.to_csv(output_cik_file, index=False)

print(f"✅ Saved {len(cik_unique_df):,} unique CIKs to {output_cik_file}")

In [ ]:
#we want to filter only IPOs which started in 2009 and forward as our SEC filings dataset only goes back this far

import pandas as pd

in_path = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_output_EODHD.csv"

# read, parsing the start_date column as datetime
df = pd.read_csv(in_path, parse_dates=["start_date"])

# filter for dates in 2009 or later
df_2009_on = df[df["start_date"] >= "2009-01-01"]

# write out filtered file
out_path = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_2009on.csv"
df_2009_on.to_csv(out_path, index=False)

print(f"Filtered down from {len(df)} rows to {len(df_2009_on)} rows. Saved to:\n{out_path}")


In [ ]:
#Now with our final filtered ticker file we add in the CIK

import pandas as pd

# Load datasets
ipo_2009_df = pd.read_csv('/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_2009on.csv')
ticker_union_df = pd.read_csv('/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_union_unique.csv')

# Merge on 'ticker'
merged_df = pd.merge(ipo_2009_df, ticker_union_df, on='ticker', how='inner')

# Count distinct CIKs
print(f"✅ After join, merged dataset has {len(merged_df)} rows and {merged_df['cik'].nunique()} distinct CIKs.")

# Save merged dataset
merged_df.to_csv('/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_2009on_with_CIK.csv', index=False)

In [ ]:
#If a company had more than one active ticker we chose the youngest one such that we can pull additional data on company filings

import pandas as pd

# Load your merged dataset
merged_df = pd.read_csv('/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_2009on_with_CIK.csv')

# Make sure start_date is parsed as datetime
merged_df['start_date'] = pd.to_datetime(merged_df['start_date'])

# Sort by CIK and start_date (latest first)
merged_sorted = merged_df.sort_values(['cik', 'start_date'], ascending=[True, False])

# For each CIK, keep the first (latest start_date)
cik_youngest_df = merged_sorted.drop_duplicates(subset='cik', keep='first')

# Reset index
cik_youngest_df = cik_youngest_df.reset_index(drop=True)

# Report
print(f"✅ After filtering for youngest ticker, {len(cik_youngest_df)} unique CIKs remain.")

# Save it
cik_youngest_df.to_csv('/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_youngest_CIK_only.csv', index=False)


In [ ]:
#adding fed data to Ticker data to get Macro economic variables

# code below gets inflation rates and Interest rates from FRED

#CPI (Consumer Price Index, All Urban Consumers, U.S. city average; CPIAUCSL):
#https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPIAUCSL

#Federal Funds Rate (Effective; FEDFUNDS):
#https://fred.stlouisfed.org/graph/fredgraph.csv?id=FEDFUNDS

import pandas as pd

# ── Step 1: load your cleaned IPO file ─────────────────────────────────────
file_path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/'
infile   = file_path + '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_youngest_CIK_only.csv' #'ticker_metrics_2009on_first_CIK_only.csv'
df = pd.read_csv(infile, parse_dates=['start_date'])

# ── Step 2: load CPI & Fed Funds, using index_col=0 ─────────────────────
cpi = pd.read_csv(
    file_path + 'CPIAUCSL.csv',
    index_col=0,
    parse_dates=True,
    comment='#'           # ignore any metadata lines
)
cpi.index.name = 'DATE'
cpi.columns = ['CPI']    # rename column to a simple name

fed = pd.read_csv(
    file_path + 'FEDFUNDS.csv',
    index_col=0,
    parse_dates=True,
    comment='#'
)
fed.index.name = 'DATE'
fed.columns = ['FEDFUNDS']

# ── preprocess CPI → YoY inflation % ───────────────────────────────────
cpi = cpi.sort_index()
cpi['YoY_inflation'] = cpi['CPI'].pct_change(12) * 100

# ── preprocess Fed Funds → forward‐fill daily rates ────────────────────
fed = fed.sort_index().ffill()

# ── Step 3: lookup function ────────────────────────────────────────────
def get_inflation_and_fed_rate(ipo_date):
    # pick last CPI on or before the IPO
    cpi_date = cpi.index[cpi.index <= ipo_date].max()
    infl     = cpi.loc[cpi_date, 'YoY_inflation']

    # pick last Fed Funds rate on or before the IPO
    fed_date = fed.index[fed.index <= ipo_date].max()
    fedr     = fed.loc[fed_date, 'FEDFUNDS']

    return round(infl, 2), round(fedr, 2)

# ── Step 4: apply to your DataFrame ────────────────────────────────────
df[['inflation_rate_at_start', 'fed_funds_rate_at_start']] = \
    df['start_date'].apply(lambda dt: pd.Series(get_inflation_and_fed_rate(dt)))

# ── Step 5: save back to Drive ────────────────────────────────────────
outfile = file_path + 'EODHD_full_ticker_data_with_adjusted_prices_with_macro.csv'
df.to_csv(outfile, index=False)
print(f"✅ Done! saved {len(df)} rows → {outfile}")


In [ ]:
#adding SnP500 Macro Data to Ticker file
import pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final")

# ── 1. Load IPO file (and be 100 % sure start_date is datetime) ─────────────
ipo = pd.read_csv(BASE / "EODHD_full_ticker_data_with_adjusted_prices_with_macro.csv",
                  parse_dates=["start_date"])
ipo["start_date"] = pd.to_datetime(ipo["start_date"], errors="coerce")

# ── 2. A *robust* loader for the S&P-500 CSV ────────────────────────────────
def load_sp500(p: Path) -> pd.DataFrame:
    # Read the CSV, skipping the first 2 rows (metadata: "Ticker,^GSPC" and "Date,NaN")
    df = pd.read_csv(p, skiprows=2, names=['Date', 'SP500_Close'])

    # Print raw data for debugging
    print("\n--- Raw S&P 500 Data (First 10 rows) ---")
    print(df.head(10))

    # Convert Date to datetime, specify format to avoid warnings
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')

    # Drop rows where Date is NaT (invalid dates, e.g., "Date")
    invalid_rows = df['Date'].isna()
    if invalid_rows.any():
        print("\n--- Invalid S&P 500 Date Rows ---")
        print(df[invalid_rows][['Date', 'SP500_Close']])
        df = df[~invalid_rows]

    # Set Date as index
    df.set_index('Date', inplace=True)

    # Ensure SP500_Close is numeric
    df['SP500_Close'] = pd.to_numeric(df['SP500_Close'], errors='coerce')

    # Drop any rows with NaN in SP500_Close
    df = df.dropna(subset=['SP500_Close'])

    # Calculate moving averages
    df['SMA_30_centered'] = df['SP500_Close'].rolling(window=31, center=True).mean()
    df['SMA_50'] = df['SP500_Close'].rolling(window=50, min_periods=1).mean()
    df['SMA_200'] = df['SP500_Close'].rolling(window=200, min_periods=1).mean()

    # Round all numeric columns to 2 decimal places
    numeric_columns = ['SP500_Close', 'SMA_30_centered', 'SMA_50', 'SMA_200']
    df[numeric_columns] = df[numeric_columns].round(2)

    # Sort index to ensure chronological order
    df = df.sort_index()

    # Validate data
    print("\n✅ S&P 500 Data Loaded.")
    print("\n--- Columns ---")
    print(df.columns.tolist())
    print("\n--- First 5 rows ---")
    print(df.head())
    print("\n--- Last 5 rows ---")
    print(df.tail())
    print("\n--- Date range ---")
    print(f"Min: {df.index.min()}, Max: {df.index.max()}")
    print("\n--- NaN counts ---")
    print(df.isna().sum())

    return df

sp500 = load_sp500(BASE / "sp500_full_yahoo.csv")

# ── 3. Helper to grab the S&P snapshot on IPO day ───────────────────────────
def sp500_on(date: pd.Timestamp) -> pd.Series:
    # Ensure date is a Timestamp
    if not isinstance(date, pd.Timestamp):
        return pd.Series({
            "sp500_close_at_start": None,
            "sp500_sma30_centered_at_start": None,
            "sp500_sma50_at_start": None,
            "sp500_sma200_at_start": None,
            "sp500_above_30sma_centered": None,
            "sp500_above_50sma": None,
            "sp500_above_200sma": None,
        })

    # Find the closest date in S&P 500 data
    loc = sp500.index.searchsorted(date, side="right") - 1
    if loc < 0 or loc >= len(sp500):
        # IPO date is outside S&P 500 data range
        print(f"Warning: Date {date} is outside S&P 500 data range ({sp500.index.min()} to {sp500.index.max()})")
        return pd.Series({
            "sp500_close_at_start": None,
            "sp500_sma30_centered_at_start": None,
            "sp500_sma50_at_start": None,
            "sp500_sma200_at_start": None,
            "sp500_above_30sma_centered": None,
            "sp500_above_50sma": None,
            "sp500_above_200sma": None,
        })

    row = sp500.iloc[loc]
    return pd.Series({
        "sp500_close_at_start": row.SP500_Close,
        "sp500_sma30_centered_at_start": row.SMA_30_centered,
        "sp500_sma50_at_start": row.SMA_50,
        "sp500_sma200_at_start": row.SMA_200,
        "sp500_above_30sma_centered": row.SP500_Close > row.SMA_30_centered if pd.notna(row.SMA_30_centered) else None,
        "sp500_above_50sma": row.SP500_Close > row.SMA_50 if pd.notna(row.SMA_50) else None,
        "sp500_above_200sma": row.SP500_Close > row.SMA_200 if pd.notna(row.SMA_200) else None,
    })

# ── 4. Enrich IPO DataFrame and show the very first row ────────────────────
ipo = pd.concat([ipo, ipo["start_date"].apply(sp500_on)], axis=1)

print("\n--- Quick preview of first row ---")
print(ipo.iloc[0])

# ── 5. If everything looks good, overwrite the file ────────────────────────
ipo.to_csv(BASE / "EODHD_full_ticker_data_with_adjusted_prices_with_macro_snp.csv", index=False)

In [ ]:
#using OpenCorperates

# — Imports —
import pandas as pd
import requests
import time
import re
from urllib.parse import quote
from google.colab import drive

# — 1) Mount Drive and Load —
drive.mount('/content/drive')
path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_full_ticker_data_with_adjusted_prices_with_macro_snp.csv' #EODHD_adjusted_macrosnp_joined_with_metadata_wsf.csv
df = pd.read_csv(path)
df.columns = df.columns.str.strip()

# Pull ALL company names
company_names = df['company_name'].astype(str).dropna().tolist()

# — 2) Cleaning Helper —
def clean_name(name):
    name = re.sub(r'\s*/.*$', '', name)   # remove "/STATE/" stuff
    name = re.sub(
        r',?\s+(Inc\.?|Corporation|Corp\.?|LLC|L\.L\.C\.|Co|Company|Ltd\.?|Trust|Group|Holdings)$',
        '',
        name,
        flags=re.IGNORECASE
    ).strip()
    return name

# — 3) OpenCorporates Lookup —
API_TOKEN = 'API Key'  # <<-- replace this
results = []

for idx, raw in enumerate(company_names):
    query = clean_name(raw)
    url   = (
        f'https://api.opencorporates.com/v0.4/companies/search'
        f'?q={quote(query)}'
        f'&api_token={API_TOKEN}'
    )
    try:
        r = requests.get(url)
        r.raise_for_status()
        comps = r.json()['results']['companies']
        incorp = comps[0]['company'].get('incorporation_date') if comps else None
    except Exception as e:
        print(f"Error for '{raw}': {e}")
        incorp = None

    results.append({
        'raw_name': raw,
        'search_name': query,
        'incorporation_date': incorp
    })

    if idx % 10 == 0:
        print(f"Processed {idx}/{len(company_names)} companies...")

    time.sleep(0.5)  # <= 2 requests per second

# — 4) Save Output —
out_df = pd.DataFrame(results)
save_path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/IPO_companies_with_incorp_dates.csv'
out_df.to_csv(save_path, index=False)
print("✅ Done! Saved to:", save_path)


In [ ]:
#See list of compnay names with Incorpdates
import pandas as pd

# Load the CSV file
default_path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/IPO_companies_with_incorp_dates.csv'
df = pd.read_csv(default_path)

# Display the first few rows
display(df.head())

# Count nulls per column
null_counts = df.isna().sum()
print("Null counts per column:")
print(null_counts)

# Total number of null values in the DataFrame
total_nulls = null_counts.sum()
print(f"\nTotal null values in the DataFrame: {total_nulls}")

# Optional: give a summary of non-null counts and dtypes
df.info()

In [ ]:
#joining the incorp dates into the EODHD ticker file
import pandas as pd

# Paths to your CSV files
incorp_path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/IPO_companies_with_incorp_dates.csv'
meta_path   = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_adjusted_macrosnp_joined_with_metadata_wsf.csv'

# Load the datasets
df_incorp = pd.read_csv(incorp_path)
df_meta   = pd.read_csv(meta_path)

# Perform the merge on raw_name and company_name (left join to keep all companies)
merged = pd.merge(
    df_incorp,
    df_meta,
    how='left',
    left_on='raw_name',
    right_on='company_name',
    suffixes=('_incorp','_meta')
)

# Inspect the result
print("Merged DataFrame head:")
display(merged.head())

# Count nulls in the merged frame
total_nulls = merged.isna().sum().sum()
print(f"\nTotal null values after merge: {total_nulls}")
print("Null counts per column:")
print(merged.isna().sum())

# Save the merged DataFrame to a new CSV
out_path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_adjusted_macrosnp_joined_with_metadata_incorp_wsf.csv'
merged.to_csv(out_path, index=False)
print(f"\nSaved merged dataset to: {out_path}")

# Optional: summary of dtypes and non-null counts
merged.info()

In [ ]:
import pandas as pd
import shutil
from pathlib import Path

# ── PATHS ──────────────────────────────────────────────────────────────
INPUT_FILE   = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/SEC_filings_cleaned_final10.csv'
IPO_FILE     = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ticker_metrics_youngest_CIK_only.csv'
OUTPUT_FILE  = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/SEC_filings_filtered_before_youngest_CIK_RETAINED.csv'

# ── 1. BACK-UP ANY OLD OUTPUT ─────────────────────────────────────────
out = Path(OUTPUT_FILE)
if out.exists():
    backup = out.with_suffix('.BACKUP_before_restart.csv')
    shutil.copy(out, backup)
    print(f'🔒 Existing output backed up to:\n   {backup}')

# ── 2. LOAD IPO START DATES INTO A DICT ───────────────────────────────
ipo_df = (
    pd.read_csv(IPO_FILE, usecols=['cik', 'start_date'])
      .assign(start_date=lambda d: pd.to_datetime(d['start_date']))
)
cik_start = dict(zip(ipo_df['cik'].astype(str), ipo_df['start_date']))
print(f'📥 Loaded {len(cik_start):,} CIK → start_date mappings.')

# ── 3. STREAM THE SEC FILE ────────────────────────────────────────────
CHUNK_SIZE = 2_500_000
first_write = True

for i, raw in enumerate(pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE)):
    print(f'📦 Chunk {i:,} | rows = {len(raw):,}')

    # 3a) FIX & PARSE ddate
    d = raw['ddate'].astype(str).str.replace(r'^3', '2', regex=True)
    raw['ddate'] = pd.to_datetime(d, format='%Y%m%d', errors='coerce')
    raw = raw.dropna(subset=['ddate'])

    # 3b) KEEP ONLY OUR CIKs
    raw['cik'] = raw['cik'].astype(str)
    raw = raw[raw['cik'].isin(cik_start)]

    # 3c) KEEP ONLY rows with ddate ≤ IPO start_date
    keep_mask = raw.apply(lambda r: r['ddate'] <= cik_start[r['cik']], axis=1)
    filt = raw[keep_mask]
    print(f'  → after filters: {len(filt):,} rows')

    # 3d) WRITE / APPEND
    if len(filt):
        mode   = 'w' if first_write else 'a'
        header = first_write
        filt.to_csv(OUTPUT_FILE, index=False, mode=mode, header=header)
        first_write = False

print('\n✅ Finished. Filtered “pre-IPO” data written to:')
print(f'   {OUTPUT_FILE}')


In [ ]:
#all in one Pivots data and builds filing features
# ================================================================
#      ROC + VALUE  FEATURES  •  QUARTER-AWARE  (ALL CIKs)
# ================================================================
!pip install tqdm --quiet

import pandas as pd, numpy as np, gc, sys
from tqdm import tqdm
from google.colab import files

def pf(*a, **k): print(*a, **k, flush=True)

# ----------------------------------------------------------------
# CONFIG
# ----------------------------------------------------------------
DATA_PATH = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/SEC_filings_filtered_before_youngest_CIK_RETAINED.csv'
OUT_PATH  = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_roc_value_stats_ALL_youngest_CIK_flat_Qcols.csv'

ID_VARS   = ['cik', 'ddate', 'qtrs']
WINDOWS   = {'All': None, '3Y': 3*365, '1Y': 365}   # in days

META_DROP = ['adsh','period','fy','fp','filed','accepted',
             'prevrpt','detail','nciks','aciks','filing_quarter']

# ----------------------------------------------------------------
# STEP 1 • LOAD + CLEAN
# ----------------------------------------------------------------
df = pd.read_csv(DATA_PATH, low_memory=False)
pf("STEP 1 • raw shape", df.shape)

df['ddate'] = pd.to_datetime(df['ddate'], errors='coerce')
df = df.dropna(subset=['ddate'])
df.drop(columns=[c for c in META_DROP if c in df.columns], inplace=True)

# ⬇️  capture every CIK *before* any filtering removes it
all_ciks = df['cik'].unique().tolist()          #  <-- NEW LINE
pf("STEP 1 • after cleaning", df.shape)


# ----------------------------------------------------------------
# STEP 2 • MELT (numeric tag values)
# ----------------------------------------------------------------
tag_cols = [c for c in df.columns if c not in ID_VARS]
df[tag_cols] = (df[tag_cols].apply(pd.to_numeric, errors='coerce')
                              .replace(0, np.nan))

df_long = (df.melt(id_vars=ID_VARS,
                   value_vars=tag_cols,
                   var_name='tag',
                   value_name='value')
             .dropna(subset=['value']))
pf("STEP 2 • long shape", df_long.shape)
del df; gc.collect()

# add helper for rolling windows
df_long['days_from_latest'] = (
    df_long.groupby('cik')['ddate'].transform('max') - df_long['ddate']
).dt.days

# ----------------------------------------------------------------
# STEP 3 • DAILY-ROC (mean)  by (cik, qtrs, tag)
# ----------------------------------------------------------------
def roc_stats(grp):
    if len(grp) < 2:
        return pd.Series({f'{w}_ROC': np.nan for w in WINDOWS})
    grp = grp.sort_values('ddate')
    grp['Δd'] = (grp['ddate'] - grp['ddate'].shift()).dt.days
    grp['Δv'] = grp['value'].diff()
    grp = grp[(grp['Δd'] > 0) & grp['Δv'].notna()]
    if grp.empty:
        return pd.Series({f'{w}_ROC': np.nan for w in WINDOWS})
    roc = (grp['Δv'] / grp['Δd']).replace([np.inf,-np.inf], np.nan)
    latest = grp['ddate'].max()
    out = {}
    for w,days in WINDOWS.items():
        sel = roc if days is None else roc[(latest - grp['ddate']).dt.days <= days]
        out[f'{w}_ROC'] = sel.mean()
    return pd.Series(out)

tqdm.pandas()
roc = (df_long.groupby(['cik','qtrs','tag'], group_keys=False)
             .progress_apply(roc_stats)
             .reset_index())
pf("STEP 3 • ROC table", roc.shape)

# ----------------------------------------------------------------
# STEP 4 • VALUE MEAN / MEDIAN  by (cik, qtrs, tag)
# ----------------------------------------------------------------
def val_stats(grp):
    out = {}
    for w,days in WINDOWS.items():
        sel = grp['value'] if days is None else grp.loc[grp['days_from_latest'] <= days,'value']
        out[f'{w}_mean']   = sel.mean()
        out[f'{w}_median'] = sel.median()
    return pd.Series(out)

vals = (df_long.groupby(['cik','qtrs','tag'], group_keys=False)
               .progress_apply(val_stats)
               .reset_index())
pf("STEP 4 • value table", vals.shape)
del df_long; gc.collect()

# ----------------------------------------------------------------
# STEP 5 • PIVOT  ⇒  wide tables with Q-prefix
# ----------------------------------------------------------------
def pivot_wide(src, value_cols, suffix):
    tables = []
    for col in value_cols:
        w = src.pivot(index='cik', columns=['qtrs','tag'], values=col)
        # flatten multi-index columns
        w.columns = [f"Q{int(q)}_{t}_{suffix}" for q,t in w.columns]
        w.reset_index(inplace=True)
        tables.append(w)
    return tables

roc_tables  = pivot_wide(roc,  [c for c in roc.columns if c.endswith('_ROC')], 'ROC')
val_tables  = []
for stat in vals.columns[3:]:        # skip cik,qtrs,tag
    base = '_'.join(stat.split('_')[1:])      # mean / median
    val_tables += pivot_wide(vals, [stat], base)

# ----------------------------------------------------------------
# STEP 6 • MERGE ALL  (retain every CIK)
# ----------------------------------------------------------------
master = pd.DataFrame({'cik': all_ciks})        #  <-- CHANGED
from functools import reduce
for tbl in roc_tables + val_tables:
    master = master.merge(tbl, on='cik', how='left')
pf("STEP 6 • merged shape", master.shape)


# ----------------------------------------------------------------
# STEP 7 • DROP all-NaN / all-zero / constant
# ----------------------------------------------------------------
drop = []
for col in master.columns[1:]:
    s = master[col].dropna()
    if s.empty or (s==0).all() or (s.nunique()==1 and master.shape[0]>1):
        drop.append(col)
master.drop(columns=drop, inplace=True)
pf(f"STEP 7 • dropped {len(drop)} useless cols → {master.shape}")

# ----------------------------------------------------------------
# STEP 8 • SAVE
# ----------------------------------------------------------------
master.to_csv(OUT_PATH, index=False)
pf(f"✅  Saved → {OUT_PATH}  ({master.shape[0]} rows × {master.shape[1]} cols)")
# files.download(OUT_PATH)  # uncomment if small enough


In [ ]:
# ===============================================================
#      QUARTER-AWARE   2-YEAR  ROC + VALUE   (chunked)
# ===============================================================
!pip install tqdm --quiet
import pandas as pd, numpy as np, gc
from collections import defaultdict
from tqdm import tqdm

IN   = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/SEC_filings_filtered_before_youngest_IPO_EODHD.csv'
OUT  = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_2Y_roc_value_stats_flat.csv'

CHUNK_ROWS = 8_000_000                        # ~45 GB peak on Colab Pro+
WINDOW_DAYS = 2 * 365                         # ← exactly one window
ID_VARS   = ['cik', 'qtrs', 'tag']
DTYPES    = {'cik':'int32', 'qtrs':'int8', 'value':'float32'}

# ── PASS-1 : latest filing date per CIK ─────────────────────────
latest = {}
for chunk in tqdm(pd.read_csv(IN, usecols=['cik','ddate'],
                              dtype={'cik':'int32','ddate':'string'},
                              chunksize=CHUNK_ROWS), desc='latest-pass'):
    chunk['ddate'] = pd.to_datetime(chunk['ddate'], errors='coerce')
    for r in chunk.groupby('cik')['ddate'].max().items():
        cik, dt = r
        latest[cik] = max(latest.get(cik, pd.Timestamp('1900-01-01')), dt)
del chunk; gc.collect()
print("Latest dates collected for", len(latest), "CIKs")

# ── accumulators ───────────────────────────────────────────────
roc_sum = defaultdict(float);  roc_cnt = defaultdict(int)
val_sum = defaultdict(lambda: {'mean':0.0,'median':0.0});  val_cnt = defaultdict(int)

# ── PASS-2 : chunk processing ──────────────────────────────────
use_cols = ID_VARS + ['ddate','value']
for chunk in tqdm(pd.read_csv(IN, usecols=use_cols,
                              chunksize=CHUNK_ROWS, dtype=DTYPES),
                  desc='stats-pass'):
    chunk['ddate'] = pd.to_datetime(chunk['ddate'], errors='coerce')
    chunk = chunk.dropna(subset=['value','ddate','tag'])
    chunk['tag'] = chunk['tag'].astype('category')

    for (cik,q,tag), grp in chunk.groupby(ID_VARS):
        grp = grp.sort_values('ddate')
        dd  = grp['ddate'].diff().dt.days
        dv  = grp['value'].diff()
        ok  = (dd>0) & dv.notna()
        roc = (dv/dd)[ok].replace([np.inf,-np.inf], np.nan).dropna()

        win_start = latest[cik] - pd.Timedelta(days=WINDOW_DAYS)
        mask_win  = grp['ddate'][ok] >= win_start

        # ----- ROC mean over 2-year window ----------------------
        if len(roc[mask_win]):
            key = (cik,q,tag,'ROC')
            roc_sum[key] += roc[mask_win].sum()
            roc_cnt[key] += roc[mask_win].count()

        # ----- value mean + median over 2-year window ----------
        sel = grp.loc[grp['ddate'] >= win_start, 'value']
        if len(sel):
            key = (cik,q,tag)
            val_sum[key]['mean']   += sel.mean()
            val_sum[key]['median'] += sel.median()
            val_cnt[key]           += 1
    del chunk, grp; gc.collect()

# ── build wide rows (one per CIK) ──────────────────────────────
rows = defaultdict(dict)

# --- ROC ---
for key, s in roc_sum.items():           # key = (cik, q, tag, 'ROC')
    cik, q, tag, _ = key
    cnt = roc_cnt[key]
    rows[cik][f"Q{q}_{tag}_2Y_ROC"] = s / cnt

# --- mean & median ---
for key, s in val_sum.items():           # key = (cik, q, tag)
    cik, q, tag = key
    if val_cnt[key]:
        rows[cik][f"Q{q}_{tag}_2Y_mean"]   = s['mean']   / val_cnt[key]
        rows[cik][f"Q{q}_{tag}_2Y_median"] = s['median'] / val_cnt[key]


wide = pd.DataFrame.from_dict(rows, orient='index').reset_index(names='cik')
print("Raw wide shape:", wide.shape)

# ── drop all-NaN/zero/constant cols ────────────────────────────
keep = ['cik']
for c in wide.columns[1:]:
    s = wide[c].dropna()
    if s.empty or (s==0).all(): continue
    if wide.shape[0] > 1 and s.nunique()==1: continue
    keep.append(c)
wide = wide[keep]
print("After pruning useless cols:", wide.shape)

# ── save ───────────────────────────────────────────────────────
wide.to_csv(OUT, index=False)
print("✅ saved", OUT)


In [ ]:
# ================================================================
#  IF CLEAN & PRUNE  →  MISSINGNESS THRESHOLDS --NOT Currently USED
# ================================================================
!pip install tqdm --quiet

import pandas as pd, numpy as np, sys
from google.colab import files

def pf(*args, **kwargs):
    print(*args, **kwargs, flush=True)

# ----------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------
INPUT_PATH   = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_roc_value_stats_ALL_CIK_flat_Qcols.csv'
OUTPUT_PATH  = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_roc_value_stats_ALL_CIK_flat_Qcols50.csv'

COL_MISS_TH  = 0.50    # drop columns with >50% missing
ROW_MISS_TH  = 1   # drop rows   with >100% missing
IMPUTE_FILL  = True    # if True, fill remaining NaNs with column median

# ----------------------------------------------------------------
# STEP 1 • LOAD
# ----------------------------------------------------------------
df = pd.read_csv(INPUT_PATH, low_memory=False)
pf(f"STEP 1 • Loaded data: {df.shape[0]:,} rows × {df.shape[1]:,} cols")

# ----------------------------------------------------------------
# STEP 2 • DROP COLUMNS BY MISSINGNESS
# ----------------------------------------------------------------
col_frac = df.isna().mean()
cols_to_drop = col_frac[col_frac > COL_MISS_TH].index.tolist()
pf(f"STEP 2 • Columns > {COL_MISS_TH*100:.0f}% missing: {len(cols_to_drop)} will be dropped")
pf("    Examples:", cols_to_drop[:10], "…")
df.drop(columns=cols_to_drop, inplace=True)
pf(f"STEP 2 • New shape: {df.shape[0]:,} rows × {df.shape[1]:,} cols")

# ----------------------------------------------------------------
# STEP 3 • DROP ROWS BY MISSINGNESS
# ----------------------------------------------------------------
row_frac = df.isna().mean(axis=1)
rows_to_drop = row_frac[row_frac > ROW_MISS_TH].index
pf(f"STEP 3 • Rows > {ROW_MISS_TH*100:.0f}% missing: {len(rows_to_drop):,} will be dropped")
df.drop(index=rows_to_drop, inplace=True)
pf(f"STEP 3 • New shape: {df.shape[0]:,} rows × {df.shape[1]:,} cols")

# ----------------------------------------------------------------
# STEP 4 • OPTIONAL IMPUTATION
# ----------------------------------------------------------------
remain_miss = int(df.isna().sum().sum())
pf(f"STEP 4 • Remaining missing values: {remain_miss:,}")
if IMPUTE_FILL and remain_miss > 0:
    medians = df.median(numeric_only=True)
    df.fillna(medians, inplace=True)
    pf("STEP 4 • Filled remaining NaNs with column medians")

# ----------------------------------------------------------------
# STEP 5 • SAVE & DOWNLOAD
# ----------------------------------------------------------------
df.to_csv(OUTPUT_PATH, index=False)
pf(f"✅  Cleaned data saved to: {OUTPUT_PATH}  ({df.shape[0]:,}×{df.shape[1]:,})")
files.download(OUTPUT_PATH)


In [ ]:
import pandas as pd

# 1. Load both files
df_roc   = pd.read_csv('/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_roc_value_stats_ALL_CIK_flat_Qcols50.csv')
df_macro = pd.read_csv('/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_adjusted_macrosnp_joined_with_metadata_incorp_wsf.csv')

# 2. Inspect shapes
print("ROC file shape:  ", df_roc.shape)
print("Macro file shape:", df_macro.shape)

# 3. Find common columns
common = list(set(df_roc.columns).intersection(df_macro.columns))
print("Common columns:\n", common)

# → pick your join‐keys from that list.
#    e.g. ticker + quarter/date column, or whatever makes sense for your data.
#    For demonstration, let's assume you want to join on 'ticker' and 'fyearq'
join_keys = ['cik']  # ← replace with your actual keys

# 4. Perform the inner join
df_merged = pd.merge(
    df_roc,
    df_macro,
    on=join_keys,
    how='inner',
    validate='one_to_many'  # optional sanity check: change if necessary
)

print("Merged shape:", df_merged.shape)

# 5. (Optional) inspect a few rows
print(df_merged.head())

# 6. Save back to CSV
output_path = '/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/EODHD_Final_cik50_flat_wsf_incorp.csv'
df_merged.to_csv(output_path, index=False)
print("Saved merged file to:", output_path)


In [ ]:
import pandas as pd

df = pd.read_csv('EODHD_Final_cik50_flat_wsf_incorp.csv')

# Convert start_date to datetime
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')

# Extract just the month in MM format
df['start_month'] = df['start_date'].dt.strftime('%m')

# Preview the result
print(df[['start_date', 'start_month']].head())

# Save to a new CSV (optional)
df.to_csv('EODHD_Final_cik50_flat_wsf_incorp_m.csv', index=False)


In [ ]:
# Modeling ran on SOL computer

# ============================================================
#  LOW-RAM GPU/CPU TRAINING  – SOL / scratch/ghtillem
#  • numeric down-cast
#  • sparse One-Hot
#  • 3-fold CV, 20 Optuna trials
# ============================================================

import os, gc, warnings, joblib, numpy as np, pandas as pd, optuna
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb, xgboost as xgb
warnings.filterwarnings("ignore")

# ---------- CONFIG ---------------------------------------------------------
DATA_FILE = "/scratch/ghtillem/EODHD_Final_maxcik50_flat_wsf_incorp_m.csv"
SAVE_DIR  = "/scratch/ghtillem/Saved_Models"
TARGETS   = ['flag_6m_price', 'flag_3y_price', 'flag_6m_avg', 'flag_3y_avg']
TEST_SIZE = 0.30
SEED      = 42
N_TRIALS  = 20     # fewer trials -> less RAM
FOLDS     = 3      # 3-fold CV
# ---------------------------------------------------------------------------

LEAKAGE = [  # full list
    'six_month_price','three_year_price','six_month_30d_avg','three_year_30d_avg',
    'flag_6m_price','flag_3y_price','flag_6m_avg','flag_3y_avg',
    'six_month_date','three_year_date','sp500_sma30_centered_at_start',
    'sp500_above_30sma_centered','_merge','cik','title','1Y_zipma_ROC',
    'All_zipba_ROC','1Y_zipba_ROC','All_sic_ROC','1Y_fye_ROC','1Y_sic_ROC',
    '3Y_zipma_ROC','3Y_zipba_ROC','All_median_zipba','Q1_zipba_median',
    'Q1_sic_mean_y','Q4_sic_median_y','Q1_zipma_median_y','Q2_sic_mean_x',
    'Q2_sic_median','Q0_sic_mean','Q0_zipba_mean','sic','Q1_zipma_median_x',
    'Q1_zipma_mean_x','Q1_fye_median_x','Q0_fye_mean','Q1_zipba_median_y',
    'Q0_zipma_ROC_x','Q4_sic_mean_y','Q0_zipba_mean_y','Q0_zipba_mean_x',
    'Q4_zipma_median_x','Q2_fye_mean_x','bus_zip','Q4_fye_median',
    'Q2_fye_mean_y','Q1_fye_mean','Q0_zipma_median_y','Q1_fye_median_y',
    'Q0_fye_median_y'
]

ID_COLS = ['ticker','company_name','phone','adsh','instance']

# ---------- 1. LOAD & DOWN-CAST -------------------------------------------
df = pd.read_csv(DATA_FILE, low_memory=False)

for col in df.select_dtypes("float"):
    df[col] = pd.to_numeric(df[col], downcast="float")
for col in df.select_dtypes("integer"):
    df[col] = pd.to_numeric(df[col], downcast="integer")

for b in ('sp500_above_50sma','sp500_above_200sma'):
    if b in df: df[b] = df[b].fillna(False).astype(int)

# ---------- 2. PREPROCESSOR -----------------------------------------------
def make_preprocessor(X):
    num = X.select_dtypes("number").columns.tolist()
    cat = X.select_dtypes("object").columns.tolist()
    return ColumnTransformer(
        transformers=[
            ("num", SimpleImputer(strategy="median"), num),
            ("cat", Pipeline([
                ("imp", SimpleImputer(strategy="constant", fill_value="missing")),
                ("ohe", OneHotEncoder(handle_unknown="ignore", sparse=True))
            ]), cat)
        ],
        sparse_threshold=0.3  # keep whole output sparse if >=30 % zeros
    )

# ---------- 3. MODEL BUILDERS (smaller) -----------------------------------
def build_lgb(t):
    return lgb.LGBMClassifier(
        device_type="gpu",
        n_estimators   = t.suggest_int ("n",     200, 800),
        num_leaves     = t.suggest_int ("leaves", 15, 128),
        max_depth      = t.suggest_int ("depth", -1, 6),
        max_bin        = 255,
        learning_rate  = t.suggest_float("lr", 3e-3, .15, log=True),
        subsample      = t.suggest_float("sub", .6, 1.0),
        colsample_bytree=t.suggest_float("col", .6, 1.0),
        random_state=SEED, n_jobs=4
    )

def build_xgb(t):
    return xgb.XGBClassifier(
        tree_method="gpu_hist", predictor="gpu_predictor",
        n_estimators   = t.suggest_int ("n",    200, 800),
        max_depth      = t.suggest_int ("depth",3, 6),
        max_bin        = 256,
        learning_rate  = t.suggest_float("lr", 3e-3, .15, log=True),
        subsample      = t.suggest_float("sub", .6, 1.0),
        colsample_bytree=t.suggest_float("col", .6, 1.0),
        reg_lambda     = t.suggest_float("l2", 1e-3, 5, log=True),
        reg_alpha      = t.suggest_float("l1", 1e-3, 5, log=True),
        random_state=SEED, n_jobs=4
    )

# ---------- 4. TRAIN LOOP --------------------------------------------------
os.makedirs(SAVE_DIR, exist_ok=True)
AUC   = make_scorer(roc_auc_score, needs_proba=True)
CV    = StratifiedKFold(FOLDS, shuffle=True, random_state=SEED)

for target in TARGETS:
    print(f"\n════ {target} ════")
    y = df[target].astype(int)
    X = (df
         .drop(columns=LEAKAGE + [target] + ID_COLS, errors="ignore")
         .assign(**{c: df[c].astype(str) for c in df.select_dtypes("object")})
    )

    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=TEST_SIZE, stratify=y, random_state=SEED)
    pre = make_preprocessor(X_tr)

    def objective(trial):
        algo = trial.suggest_categorical("algo", ["lgb", "xgb"])
        clf  = build_lgb(trial) if algo=="lgb" else build_xgb(trial)
        pipe = Pipeline([("prep", pre), ("clf", clf)])
        score = cross_val_score(pipe, X_tr, y_tr, cv=CV,
                                scoring=AUC, n_jobs=1).mean()
        return 0.5 if np.isnan(score) else score

    study = optuna.create_study(direction="maximize",
                                sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)

    params = study.best_trial.params; algo = params.pop("algo")
    best_clf = build_lgb(optuna.trial.FixedTrial(params)) if algo=="lgb" else \
               build_xgb(optuna.trial.FixedTrial(params))

    pipe = Pipeline([("prep", pre), ("clf", best_clf)])
    pipe.fit(X_tr, y_tr)

    auc = roc_auc_score(y_te, pipe.predict_proba(X_te)[:,1])
    print(f"  • test AUC = {auc:.4f}  |  {algo.upper()}")

    path = f"{SAVE_DIR}/{target}_lowRAM.pkl"
    joblib.dump(pipe, path, compress=3)
    print("  💾 saved", path)

    del pipe, pre, best_clf; gc.collect()

print("\n✅ Finished.")


In [ ]:
# NN model ran on SOL

import os
try:
    import tensorflow as tf
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
        print(f"Enabled TF GPU memory growth for {len(gpus)} GPU(s).")
    else:
        print("No GPU detected by TensorFlow.")
except ImportError:
    print("TensorFlow not found, skipping GPU memory growth setting.")

import json
import warnings
import joblib
from datetime import datetime
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
import optuna

# --- CONFIG ---
DATA_FILE         = '/scratch/ghtillem/Data/EODHD_Final_maxcik50_flat_wsf_incorp_m.csv'
OUTPUT_DIR        = '/scratch/ghtillem/Saved_NN_Models/final/final'
os.makedirs(OUTPUT_DIR, exist_ok=True)

TEST_SIZE         = 0.30
VAL_FOLDS         = 5
RANDOM_SEED       = 42
BATCH_SIZE        = 512
MAX_EPOCHS        = 100
EARLY_PATIENCE    = 25
N_TRIALS          = 100 # Increased for proper hyperparameter search
CLEAN_OLD_STUDIES = False

TARGETS = ['flag_3y_avg', 'flag_6m_avg']
LEAKAGE = [
    'six_month_price', 'three_year_price', 'six_month_30d_avg', 'three_year_30d_avg',
    'flag_6m_price', 'flag_3y_price', 'six_month_date', 'three_year_date',
    'sp500_sma30_centered_at_start', 'sp500_above_30sma_centered', '_merge',
    'cik', 'title', '1Y_zipma_ROC', 'All_zipba_ROC', '1Y_zipba_ROC', 'All_sic_ROC',
    '1Y_fye_ROC', '1Y_sic_ROC', '3Y_zipma_ROC', '3Y_zipba_ROC', 'All_median_zipba',
    'Q1_zipba_median', 'Q1_sic_mean_y', 'Q4_sic_median_y', 'Q1_zipma_median_y',
    'Q2_sic_mean_x', 'Q2_sic_median', 'Q0_sic_mean', 'Q0_zipba_mean', 'sic',
    'Q1_zipma_median_x', 'Q1_zippa_mean_x', 'Q1_fye_median_x', 'Q0_fye_mean',
    'Q1_zipba_median_y', 'Q0_zipma_ROC_x', 'Q4_sic_mean_y', 'Q0_zipba_mean_y',
    'Q0_zipba_mean_x', 'Q4_zipma_median_x', 'Q2_fye_mean_x', 'bus_zip', 'Q4_fye_median',
    'Q2_fye_mean_y', 'Q1_fye_mean', 'Q0_zipma_median_y', 'Q1_fye_median_y', 'Q0_fye_median_y',
    'phone', 'ticker', 'start_date'
]

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Preprocessing Function ---
def build_preprocessor(df, target_cols_to_drop, include_pca=False, pca_n=None):
    """
    Builds the preprocessing pipeline. Optionally includes PCA if specified.
    Excludes leakage and specified target columns.
    Returns the preprocessor and lists of numerical and categorical columns.
    """
    cols_to_drop = set(LEAKAGE) | set(target_cols_to_drop)
    df_features = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

    feature_cols = df_features.columns.tolist()
    if not feature_cols:
        raise ValueError("No feature columns remaining after dropping leakage and targets.")

    num_cols = df_features.select_dtypes(include=np.number).columns.tolist()
    cat_cols = df_features.select_dtypes(exclude=np.number).columns.tolist()
    logger.debug(f"Num cols identified: {len(num_cols)}")
    logger.debug(f"Cat cols identified: {len(cat_cols)}")

    num_pipe = Pipeline([
        ('impute', SimpleImputer(strategy='median')),
        ('scale', StandardScaler())
    ])
    if include_pca and pca_n is not None and pca_n > 0:
        num_pipe.steps.append(('pca', PCA(n_components=pca_n)))
        logger.info(f"Added PCA with n_components={pca_n} to numerical pipeline.")

    cat_pipe = Pipeline([
        ('impute', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer([
        ('num', num_pipe, num_cols),
        ('cat', cat_pipe, cat_cols)
    ], remainder='drop')

    return preprocessor, num_cols, cat_cols

# --- Save Preprocessing Metadata ---
def save_preprocessing_metadata(output_dir, target, num_cols, cat_cols, preprocessor, timestamp):
    """
    Saves preprocessing metadata (numerical and categorical columns, feature names after OHE).
    """
    metadata = {
        'numerical_columns': num_cols,
        'categorical_columns': cat_cols,
    }
    try:
        # Get feature names after one-hot encoding
        ohe = preprocessor.named_transformers_['cat'].named_steps['ohe']
        cat_feature_names = ohe.get_feature_names_out(metadata['categorical_columns']).tolist()
        # Numerical feature names (after PCA if applicable)
        if 'pca' in preprocessor.named_transformers_['num'].named_steps:
            num_feature_names = [f'pca_{i}' for i in range(preprocessor.named_transformers_['num'].named_steps['pca'].n_components_)]
        else:
            num_feature_names = metadata['numerical_columns']
        metadata['feature_names'] = num_feature_names + cat_feature_names
    except Exception as e:
        logger.warning(f"Could not retrieve feature names: {e}")
        metadata['feature_names'] = []

    metadata_path = os.path.join(output_dir, f"{target}_preprocessor_metadata_{timestamp}.json")
    try:
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        logger.info(f"Saved preprocessing metadata to {metadata_path}")
    except Exception as e:
        logger.error(f"Error saving preprocessing metadata: {e}")

# --- Load Data ---
logger.info(f"Loading data from {DATA_FILE}")
try:
    df = pd.read_csv(DATA_FILE, low_memory=False)
    logger.info(f"Data loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    logger.error(f"Data file not found at {DATA_FILE}")
    sys.exit(1)
except Exception as e:
    logger.error(f"Error loading data: {e}")
    sys.exit(1)

# --- Three-Way Split for Proper Validation ---
logger.info("Performing three-way split (train/validation/test)...")
results = []
all_best_params = {}

for TARGET in TARGETS:
    logger.info(f"================ Processing Target: {TARGET} ================")

    # Drop rows where the current target is missing
    df_t = df.dropna(subset=[TARGET]).copy()
    if df_t.empty:
        logger.warning(f"No data remaining for target {TARGET} after dropping NaNs. Skipping.")
        continue
    df_t.reset_index(drop=True, inplace=True)
    y = df_t[TARGET].astype(int)
    X = df_t.drop(columns=[TARGET])

    other_targets = [t for t in TARGETS if t != TARGET]

    # Three-way split: train + validation (for Optuna CV) + test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y,
        test_size=TEST_SIZE,
        stratify=y,
        random_state=RANDOM_SEED
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=0.2,  # 20% of remaining data for validation
        stratify=y_temp,
        random_state=RANDOM_SEED
    )
    X_trainval = X_temp  # Full train+val set for final training
    y_trainval = y_temp
    logger.info(f"Train set size: {X_train.shape[0]}, Validation set size: {X_val.shape[0]}, Test set size: {X_test.shape[0]}")

    # Log class distribution
    logger.info(f"Class distribution in y_train for {TARGET}: {np.bincount(y_train)}")
    logger.info(f"Class distribution in y_val for {TARGET}: {np.bincount(y_val)}")
    logger.info(f"Class distribution in y_test for {TARGET}: {np.bincount(y_test)}")

    # --- Build Base Preprocessor ---
    base_preprocessor, num_cols, cat_cols = build_preprocessor(X_trainval, other_targets, include_pca=False)
    if not num_cols and not cat_cols:
        logger.error(f"No numerical or categorical columns identified for target {TARGET}. Check LEAKAGE list and data.")
        continue

    # --- Optuna Hyperparameter Optimization ---
    logger.info(f"Starting Optuna hyperparameter search for {TARGET}...")

    def objective(trial):
        max_pca_components = min(len(num_cols), 50) if num_cols else 0
        pca_n = trial.suggest_int('pca_n', 0, max_pca_components) if max_pca_components > 0 else 0
        n_layers = trial.suggest_int('n_layers', 1, 5)
        units = trial.suggest_int('units', 32, 512, step=32)
        dropout_rate = trial.suggest_float('dropout', 0.1, 0.5, step=0.1)
        learning_rate = trial.suggest_float('lr', 1e-5, 1e-2, log=True)

        trial_preprocessor, _, _ = build_preprocessor(
            X_trainval, other_targets, include_pca=(pca_n > 0), pca_n=pca_n
        )

        skf = StratifiedKFold(n_splits=VAL_FOLDS, shuffle=True, random_state=RANDOM_SEED)
        fold_val_aucs = []
        fold_num = 0
        for train_i, val_i in skf.split(X_train, y_train):
            fold_num += 1
            logger.debug(f"Trial {trial.number}, Fold {fold_num}/{VAL_FOLDS}")
            X_tr, X_val_fold = X_train.iloc[train_i], X_train.iloc[val_i]
            y_tr, y_val_fold = y_train.iloc[train_i], y_train.iloc[val_i]

            X_tr_p = trial_preprocessor.fit_transform(X_tr)
            X_val_p = trial_preprocessor.transform(X_val_fold)

            tf.keras.backend.clear_session()
            inp = keras.Input(shape=(X_tr_p.shape[1],))
            x = inp
            for i in range(n_layers):
                x = layers.Dense(units, activation='relu', name=f'dense_{i}')(x)
                x = layers.BatchNormalization(name=f'bn_{i}')(x)
                x = layers.Dropout(dropout_rate, name=f'dropout_{i}')(x)
            out = layers.Dense(1, activation='sigmoid', name='output')(x)
            model = keras.Model(inp, out)

            model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                loss='binary_crossentropy',
                metrics=[keras.metrics.AUC(name='auc')]
            )

            try:
                unique_classes = np.unique(y_tr)
                if len(unique_classes) > 1:
                    weights = class_weight.compute_class_weight(class_weight='balanced', classes=unique_classes, y=y_tr)
                    class_weights_dict = dict(enumerate(weights))
                else:
                    logger.warning(f"Fold {fold_num} - Only one class present. No class weights.")
                    class_weights_dict = None
            except ValueError as e:
                logger.warning(f"Fold {fold_num} - Could not compute class weights: {e}.")
                class_weights_dict = None

            early_stopping = keras.callbacks.EarlyStopping(
                monitor='val_auc',
                patience=EARLY_PATIENCE,
                mode='max',
                restore_best_weights=True
            )

            history = model.fit(
                X_tr_p, y_tr,
                validation_data=(X_val_p, y_val_fold),
                epochs=MAX_EPOCHS,
                batch_size=BATCH_SIZE,
                class_weight=class_weights_dict,
                callbacks=[early_stopping],
                verbose=0
            )

            best_val_auc = max(history.history['val_auc'])
            fold_val_aucs.append(best_val_auc)

        mean_val_auc = float(np.mean(fold_val_aucs))
        logger.info(f"Trial {trial.number} completed. Mean Validation AUC: {mean_val_auc:.4f}")
        return mean_val_auc

    study_name = f"optuna_nn_{TARGET}"
    storage_path = f"sqlite:///{os.path.join(OUTPUT_DIR, study_name)}.db"
    logger.info(f"Optuna study: {study_name}, Storage: {storage_path}")

    if CLEAN_OLD_STUDIES:
        db_file = os.path.join(OUTPUT_DIR, f"{study_name}.db")
        if os.path.exists(db_file):
            logger.warning(f"Removing existing Optuna study database: {db_file}")
            os.remove(db_file)

    study = optuna.create_study(
        study_name=study_name,
        direction='maximize',
        sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=VAL_FOLDS // 2),
        storage=storage_path,
        load_if_exists=not CLEAN_OLD_STUDIES
    )

    study.optimize(objective, n_trials=N_TRIALS, timeout=None)

    study_pkl_path = os.path.join(OUTPUT_DIR, f"{study_name}.pkl")
    joblib.dump(study, study_pkl_path)
    logger.info(f"Optuna study results saved to {study_pkl_path}")

    best_params = study.best_params
    best_value = study.best_value
    all_best_params[TARGET] = best_params
    logger.info(f"Best trial for {TARGET}: Value (Mean Val AUC): {best_value:.4f}")
    logger.info(f"Best hyperparameters for {TARGET}: {best_params}")

    # --- Final Model Training & Saving ---
    logger.info(f"Starting final model training for {TARGET}...")

    # Build final preprocessor with PCA if selected by Optuna
    best_pca_n = best_params.get('pca_n', 0)
    final_preprocessor, num_cols_final, cat_cols_final = build_preprocessor(
        X_trainval, other_targets, include_pca=(best_pca_n > 0), pca_n=best_pca_n
    )
    final_preprocessor.fit(X_trainval)

    X_train_processed = final_preprocessor.transform(X_train)
    X_val_processed = final_preprocessor.transform(X_val)
    X_test_processed = final_preprocessor.transform(X_test)
    logger.info(f"Processed shapes - Train: {X_train_processed.shape}, Val: {X_val_processed.shape}, Test: {X_test_processed.shape}")

    # Save the final fitted preprocessor and metadata
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    prep_final_path = os.path.join(OUTPUT_DIR, f"{TARGET}_preprocessor_final_{timestamp}.joblib")
    joblib.dump(final_preprocessor, prep_final_path)
    logger.info(f"Saved final fitted preprocessor to {prep_final_path}")

    save_preprocessing_metadata(OUTPUT_DIR, TARGET, num_cols_final, cat_cols_final, final_preprocessor, timestamp)

    # Build and train the final model
    tf.keras.backend.clear_session()
    input_shape = (X_train_processed.shape[1],)
    inp_f = keras.Input(shape=input_shape)
    x_f = inp_f
    best_n_layers = best_params.get('n_layers', 1)
    best_units = best_params['units']
    best_dropout = best_params['dropout']
    best_lr = best_params['lr']

    for i in range(best_n_layers):
        x_f = layers.Dense(best_units, activation='relu', name=f'dense_{i}')(x_f)
        x_f = layers.BatchNormalization(name=f'bn_{i}')(x_f)
        x_f = layers.Dropout(best_dropout, name=f'dropout_{i}')(x_f)
    out_f = layers.Dense(1, activation='sigmoid', name='output')(x_f)
    final_model = keras.Model(inp_f, out_f)

    final_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=best_lr),
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC(name='auc')]
    )
    logger.info("Final model compiled.")
    final_model.summary(print_fn=logger.info)

    try:
        unique_classes_final = np.unique(y_train)
        if len(unique_classes_final) > 1:
            weights_final = class_weight.compute_class_weight(class_weight='balanced', classes=unique_classes_final, y=y_train)
            class_weights_dict_final = dict(enumerate(weights_final))
            logger.info(f"Final class weights: {class_weights_dict_final}")
        else:
            logger.warning("Only one class present in final training data.")
            class_weights_dict_final = None
    except ValueError as e:
        logger.warning(f"Could not compute final class weights: {e}.")
        class_weights_dict_final = None

    final_early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_auc',
        patience=EARLY_PATIENCE,
        mode='max',
        restore_best_weights=True,
        verbose=1
    )

    final_model.fit(
        X_train_processed, y_train,
        validation_data=(X_val_processed, y_val),
        epochs=MAX_EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=class_weights_dict_final,
        callbacks=[final_early_stopping],
        verbose=1
    )

    # Evaluate on validation and test sets
    logger.info(f"Evaluating final model for {TARGET} on validation set...")
    val_loss, val_auc = final_model.evaluate(X_val_processed, y_val, verbose=0)
    logger.info(f"Validation Set Performance for {TARGET}: Loss = {val_loss:.4f}, AUC = {val_auc:.4f}")

    logger.info(f"Evaluating final model for {TARGET} on test set...")
    test_loss, test_auc = final_model.evaluate(X_test_processed, y_test, verbose=0)
    logger.info(f"Test Set Performance for {TARGET}: Loss = {test_loss:.4f}, AUC = {test_auc:.4f}")

    # Save the final trained model
    model_path = os.path.join(OUTPUT_DIR, f"{TARGET}_model.keras")
    final_model.save(model_path)
    logger.info(f"Saved final trained model to {model_path}")

    results.append({
        'target': TARGET,
        'val_auc': float(val_auc),
        'test_auc': float(test_auc)
    })

# --- Summary ---
logger.info("NN training process complete for all targets.")
results_df = pd.DataFrame(results)
summary_path = os.path.join(OUTPUT_DIR, 'summary_nn.csv')
results_df.to_csv(summary_path, index=False)
logger.info(f"Summary of validation and test AUCs saved to {summary_path}")
print("\n--- Final Validation and Test Set AUC Summary ---")
print(results_df)
print("---------------------------------")

logger.info(f"Best hyperparameters found: {json.dumps(all_best_params, indent=2)}")
print(f"NN training complete. Artifacts saved in: {OUTPUT_DIR}")

In [ ]:
# LightNGB ran on SOL

import os
import json
import logging
import numpy as np
import pandas as pd
import joblib
import optuna
from datetime import datetime
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

# --- CONFIG ---
DATA_FILE = '/scratch/ghtillem/Data/EODHD_Final_maxcik50_flat_wsf_incorp_m.csv'
OUTPUT_DIR = '/scratch/ghtillem/Saved_LGBM_Models/Final/final'
os.makedirs(OUTPUT_DIR, exist_ok=True)

TEST_SIZE = 0.30
VAL_FOLDS = 5
VAL_REPEATS = 2
RANDOM_SEED = 42
LGBM_EARLY_STOPPING_ROUNDS = 50
N_TRIALS = 100
CLEAN_OLD_STUDIES = False

TARGETS = ['flag_3y_avg', 'flag_6m_avg']
LEAKAGE = [
    'six_month_price', 'three_year_price', 'six_month_30d_avg', 'three_year_30d_avg',
    'flag_6m_price', 'flag_3y_price', 'six_month_date', 'three_year_date',
    'sp500_sma30_centered_at_start', 'sp500_above_30sma_centered', '_merge',
    'cik', 'title', '1Y_zipma_ROC', 'All_zipba_ROC', '1Y_zipba_ROC', 'All_sic_ROC',
    '1Y_fye_ROC', '1Y_sic_ROC', '3Y_zipma_ROC', '3Y_zipba_ROC', 'All_median_zipba',
    'Q1_zipba_median', 'Q1_sic_mean_y', 'Q4_sic_median_y', 'Q1_zipma_median_y',
    'Q2_sic_mean_x', 'Q2_sic_median', 'Q0_sic_mean', 'Q0_zipba_mean', 'sic',
    'Q1_zipma_median_x', 'Q1_zippa_mean_x', 'Q1_fye_median_x', 'Q0_fye_mean',
    'Q1_zipba_median_y', 'Q0_zipma_ROC_x', 'Q4_sic_mean_y', 'Q0_zipba_mean_y',
    'Q0_zipba_mean_x', 'Q4_zipma_median_x', 'Q2_fye_mean_x', 'bus_zip', 'Q4_fye_median',
    'Q2_fye_mean_y', 'Q1_fye_mean', 'Q0_zipma_median_y', 'Q1_fye_median_y', 'Q0_fye_median_y',
    'phone', 'ticker', 'start_date'
]

# Setup logging
timestamp_fmt = '%Y%m%d_%H%M%S'
logging.basicConfig(
    filename=os.path.join(OUTPUT_DIR, 'training.log'),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

# Detect GPU support
device_type = 'cpu'
try:
    lgb.LGBMClassifier(device_type='gpu').fit(np.array([[1]]), [1])
    device_type = 'gpu'
    logger.info("GPU support confirmed.")
except Exception:
    logger.info("GPU not available, using CPU.")

# --- Global Clip Outliers Function ---
def clip_outliers(arr):
    """
    Clips outliers in numerical data using the IQR method.
    """
    q1, q3 = np.percentile(arr, [25, 75], axis=0)
    iqr = q3 - q1
    return np.clip(arr, q1 - 1.5 * iqr,  q3 + 1.5 * iqr)

# --- Preprocessing Function ---
def build_preprocessor(df, target_cols_to_drop):
    """
    Builds the preprocessing pipeline with outlier clipping for numerical features.
    Excludes leakage and specified target columns.
    Returns the preprocessor, numerical columns, and categorical columns.
    """
    cols_to_drop = set(LEAKAGE) | set(target_cols_to_drop)
    df_features = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

    feature_cols = df_features.columns.tolist()
    if not feature_cols:
        raise ValueError("No feature columns remaining after dropping leakage and targets.")

    num_cols = df_features.select_dtypes(include=np.number).columns.tolist()
    cat_cols = df_features.select_dtypes(exclude=np.number).columns.tolist()
    logger.info(f"Identified {len(num_cols)} numerical and {len(cat_cols)} categorical columns.")

    num_pipe = Pipeline([
        ('impute', SimpleImputer(strategy='median')),
        ('clip', FunctionTransformer(clip_outliers, validate=False)),
        ('scale', StandardScaler())
    ])

    cat_pipe = Pipeline([
        ('impute', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer([
        ('num', num_pipe, num_cols),
        ('cat', cat_pipe, cat_cols)
    ], remainder='drop')

    return preprocessor, num_cols, cat_cols

# --- Save Preprocessing Metadata ---
def save_preprocessing_metadata(output_dir, target, num_cols, cat_cols, preprocessor, timestamp):
    """
    Saves preprocessing metadata (numerical and categorical columns, feature names after OHE).
    """
    metadata = {
        'numerical_columns': num_cols,
        'categorical_columns': cat_cols,
    }
    try:
        ohe = preprocessor.named_transformers_['cat'].named_steps['ohe']
        cat_feature_names = ohe.get_feature_names_out(metadata['categorical_columns']).tolist()
        metadata['feature_names'] = num_cols + cat_feature_names
    except Exception as e:
        logger.warning(f"Could not retrieve feature names: {e}")
        metadata['feature_names'] = []

    metadata_path = os.path.join(output_dir, f"{target}_preprocessor_metadata_{timestamp}.json")
    try:
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        logger.info(f"Saved preprocessing metadata to {metadata_path}")
    except Exception as e:
        logger.error(f"Error saving preprocessing metadata: {e}")

# --- Load Data ---
logger.info(f"Loading data from {DATA_FILE}")
try:
    df = pd.read_csv(DATA_FILE, low_memory=False)
    if df.empty:
        raise ValueError("Dataframe empty.")
    missing = set(TARGETS) - set(df.columns)
    if missing:
        raise ValueError(f"Missing target columns: {missing}")
    logger.info(f"Data loaded: {df.shape}")
except Exception as e:
    logger.error(f"Error loading data: {e}")
    raise

# --- Drop Leakage Columns ---
df.drop(columns=[c for c in LEAKAGE if c not in TARGETS], errors='ignore', inplace=True)

results = []
all_best_params = {}

for TARGET in TARGETS:
    logger.info(f"================ Processing Target: {TARGET} ================")
    df_t = df.dropna(subset=[TARGET]).copy()
    if df_t.empty:
        logger.warning(f"No data remaining for target {TARGET} after dropping NaNs. Skipping.")
        continue
    df_t.reset_index(drop=True, inplace=True)

    # Convert bool to int
    bool_cols = df_t.select_dtypes(include='bool').columns
    df_t[bool_cols] = df_t[bool_cols].astype(int)

    y = df_t[TARGET].astype(int)
    X = df_t.drop(columns=[TARGET])

    # Three-way split: train, validation, test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_SEED
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.2, stratify=y_temp, random_state=RANDOM_SEED
    )
    X_trainval = X_temp
    y_trainval = y_temp
    logger.info(f"Train set size: {X_train.shape[0]}, Validation set size: {X_val.shape[0]}, Test set size: {X_test.shape[0]}")

    # Log class distribution
    logger.info(f"Class distribution in y_train for {TARGET}: {np.bincount(y_train)}")
    logger.info(f"Class distribution in y_val for {TARGET}: {np.bincount(y_val)}")
    logger.info(f"Class distribution in y_test for {TARGET}: {np.bincount(y_test)}")

    # Build preprocessor
    other_targets = [t for t in TARGETS if t != TARGET]
    preprocessor, num_cols, cat_cols = build_preprocessor(X_trainval, other_targets)
    preprocessor.fit(X_trainval)

    # Transform data
    X_train_p = preprocessor.transform(X_train)
    X_val_p = preprocessor.transform(X_val)
    X_test_p = preprocessor.transform(X_test)
    cat_indices = list(range(len(num_cols), len(num_cols) + len(cat_cols)))
    logger.info(f"Processed shapes - Train: {X_train_p.shape}, Val: {X_val_p.shape}, Test: {X_test_p.shape}")

    # Save preprocessor and metadata
    timestamp = datetime.now().strftime(timestamp_fmt)
    prep_path = os.path.join(OUTPUT_DIR, f"{TARGET}_preprocessor_{timestamp}.joblib")
    joblib.dump(preprocessor, prep_path)
    logger.info(f"Saved preprocessor to {prep_path}")

    save_preprocessing_metadata(OUTPUT_DIR, TARGET, num_cols, cat_cols, preprocessor, timestamp)

    # Optuna objective
    def objective(trial):
        params = {
            'objective': 'binary', 'metric': 'auc',
            'random_state': RANDOM_SEED, 'n_jobs': -1,
            'device_type': device_type,
            'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
            'n_estimators': trial.suggest_int('n_estimators', 200, 2500, step=100),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.01, 0.05, 0.1]),
            'num_leaves': trial.suggest_int('num_leaves', 10, 150),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10, log=True),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
        }
        skf = RepeatedStratifiedKFold(n_splits=VAL_FOLDS, n_repeats=VAL_REPEATS, random_state=RANDOM_SEED)
        aucs = []
        fold_num = 0
        for train_idx, val_idx in skf.split(X_train, y_train):
            fold_num += 1
            Xt, Xv = X_train_p[train_idx], X_train_p[val_idx]
            yt, yv = y_train.iloc[train_idx], y_train.iloc[val_idx]
            params['scale_pos_weight'] = (yt == 0).sum() / (yt == 1).sum() if (yt == 1).sum() > 0 else 1.0
            model = lgb.LGBMClassifier(**params)
            callbacks = [lgb.early_stopping(LGBM_EARLY_STOPPING_ROUNDS)] if params['boosting_type'] != 'dart' else []
            model.fit(
                Xt, yt,
                eval_set=[(Xv, yv)],
                eval_metric='auc',
                callbacks=callbacks,
                categorical_feature=cat_indices
            )
            auc = roc_auc_score(yv, model.predict_proba(Xv)[:, 1])
            aucs.append(auc)
            trial.report(np.mean(aucs), fold_num)
            if trial.should_prune():
                logger.info(f"Trial {trial.number} pruned.")
                raise optuna.exceptions.TrialPruned()
        mean_auc = float(np.mean(aucs))
        logger.info(f"Trial {trial.number} completed. Mean CV AUC: {mean_auc:.4f}")
        return mean_auc

    # Run Optuna
    study_name = f"optuna_lgbm_{TARGET}"
    db_path = os.path.join(OUTPUT_DIR, f"{study_name}.db")
    if CLEAN_OLD_STUDIES and os.path.exists(db_path):
        logger.warning(f"Removing existing Optuna study database: {db_path}")
        os.remove(db_path)
    study = optuna.create_study(
        study_name=study_name,
        direction='maximize',
        sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
        pruner=optuna.pruners.SuccessiveHalvingPruner(),
        storage=f"sqlite:///{db_path}",
        load_if_exists=not CLEAN_OLD_STUDIES
    )
    study.optimize(objective, n_trials=N_TRIALS)

    # Save Optuna results
    study_pkl_path = os.path.join(OUTPUT_DIR, f"{study_name}.pkl")
    joblib.dump(study, study_pkl_path)
    logger.info(f"Saved Optuna study to {study_pkl_path}")
    pd.DataFrame(study.trials_dataframe()).to_csv(os.path.join(OUTPUT_DIR, f"{study_name}_trials.csv"), index=False)
    with open(os.path.join(OUTPUT_DIR, f"{study_name}_best.json"), 'w') as f:
        json.dump(study.best_params, f, indent=2)

    # Final model training
    logger.info(f"Training final model for {TARGET} on full trainval set")
    final_params = study.best_params.copy()
    final_params.update({
        'objective': 'binary',
        'metric': 'auc',
        'random_state': RANDOM_SEED,
        'n_jobs': -1,
        'device_type': device_type,
        'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum() if (y_train == 1).sum() > 0 else 1.0
    })
    final_model = lgb.LGBMClassifier(**final_params)
    callbacks_final = [lgb.early_stopping(LGBM_EARLY_STOPPING_ROUNDS)] if final_params.get('boosting_type') != 'dart' else []
    final_model.fit(
        X_train_p, y_train,
        eval_set=[(X_val_p, y_val)],
        eval_metric='auc',
        callbacks=callbacks_final,
        categorical_feature=cat_indices
    )

    # Evaluate on validation and test sets
    val_auc = roc_auc_score(y_val, final_model.predict_proba(X_val_p)[:, 1])
    test_auc = roc_auc_score(y_test, final_model.predict_proba(X_test_p)[:, 1])
    logger.info(f"Validation AUC for {TARGET}: {val_auc:.4f}")
    logger.info(f"Test AUC for {TARGET}: {test_auc:.4f}")

    # Save final artifacts
    ts = datetime.now().strftime(timestamp_fmt)
    model_file = os.path.join(OUTPUT_DIR, f"{TARGET}_model_{ts}.txt")
    final_model.booster_.save_model(model_file)
    logger.info(f"Saved model to {model_file}")

    # Feature importance
    fi = pd.DataFrame({
        'feature': final_model.booster_.feature_name(),
        'importance': final_model.feature_importances_
    })
    fi.sort_values('importance', ascending=False).to_csv(
        os.path.join(OUTPUT_DIR, f"{TARGET}_feature_importance.csv"), index=False
    )

    results.append({
        'target': TARGET,
        'best_cv_auc': study.best_value,
        'val_auc': val_auc,
        'test_auc': test_auc})

# Save summary
results_df = pd.DataFrame(results)
summary_path = os.path.join(OUTPUT_DIR, 'summary_lgbm_results.csv')
results_df.to_csv(summary_path, index=False)
logger.info(f"Saved summary to {summary_path}")
print("\n--- Final Validation and Test AUC Summary ---")
print(results_df)
print("---------------------------------")

logger.info(f"Best hyperparameters found: {json.dumps(all_best_params, indent=2)}")
print(f"LGBM training complete. Artifacts saved in: {OUTPUT_DIR}")

In [ ]:
#Simulation ran on SOL

#df_test is 30% split with randomseed = 42 using  EODHD_Final_maxcik50_flat_wsf_incorp_m.csv

import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ─── 5) Simulation / Backtest ───────────────────────────────
logger.info("Starting backtest simulation...")

# --- Simulation Parameters ---
initial_capital = 100_000.0
investment_per_signal = 100.0
investment_both_signals = 200.0
prediction_threshold = 0.6

# --- Initialize Strategy Capitals ---
rnd_cap = initial_capital
mdl_cap = initial_capital
nn_cap = initial_capital
ens_cap = initial_capital

# --- Initialize Cumulative Profits and Metrics Tracking ---
rnd_cum = 0.0
mdl_cum = 0.0
nn_cum = 0.0
ens_cum = 0.0

# Metrics tracking
rnd_wins = 0
rnd_losses = 0
rnd_win_profits = []
rnd_loss_profits = []
rnd_cum_profits = [0.0]  # For max drawdown
rnd_investments = []

mdl_wins = 0
mdl_losses = 0
mdl_win_profits = []
mdl_loss_profits = []
mdl_cum_profits = [0.0]
mdl_investments = []
mdl_correct_6m = 0
mdl_correct_3y = 0
mdl_predictions_6m = 0
mdl_predictions_3y = 0

nn_wins = 0
nn_losses = 0
nn_win_profits = []
nn_loss_profits = []
nn_cum_profits = [0.0]
nn_investments = []
nn_correct_6m = 0
nn_correct_3y = 0
nn_predictions_6m = 0
nn_predictions_3y = 0

ens_wins = 0
ens_losses = 0
ens_win_profits = []
ens_loss_profits = []
ens_cum_profits = [0.0]
ens_investments = []
ens_predictions_6m = 0
ens_predictions_3y = 0
ens_correct_6m     = 0
ens_correct_3y     = 0

# --- History Tracking ---
history = []
processed_rows = 0
skipped_rows = 0

# --- Verify Prediction Arrays ---
try:
    assert len(y_prob_6m) == len(df_test), f"y_prob_6m length mismatch: {len(y_prob_6m)} vs {len(df_test)}"
    assert len(y_prob_3y) == len(df_test), f"y_prob_3y length mismatch: {len(y_prob_3y)} vs {len(df_test)}"
    assert len(y_prob_nn_6m) == len(df_test), f"y_prob_nn_6m length mismatch: {len(y_prob_nn_6m)} vs {len(df_test)}"
    assert len(y_prob_nn_3y) == len(df_test), f"y_prob_nn_3y length mismatch: {len(y_prob_nn_3y)} vs {len(df_test)}"
    logger.info("All required prediction arrays found. Proceeding with simulation.")
except AssertionError as e:
    logger.error(f"Prediction array length validation failed: {e}")
    logger.error(traceback.format_exc())
    raise RuntimeError("Prediction array validation failed") from e

# --- Simulation Loop ---
try:
    for i in range(len(df_test)):
        true_6m = y_true_6m[i]
        true_3y = y_true_3y[i]
        sp = start_prices[i]
        sp6 = sixm_prices[i]
        sp3 = threey_prices[i]
        ret_6m = p6m[i]
        ret_3y = p3y[i]

        # --- Data Validation ---
        if (pd.isna(true_6m) or pd.isna(true_3y) or
                pd.isna(sp) or sp <= 0 or
                pd.isna(sp6) or sp6 <= 0 or
                pd.isna(sp3) or sp3 <= 0 or
                pd.isna(ret_6m) or pd.isna(ret_3y) or
                ret_6m < percentile_6m_lower or ret_6m > percentile_6m_upper or
                ret_3y < percentile_3y_lower or ret_3y > percentile_3y_upper):
            skipped_rows += 1
            logger.debug(f"Skipping row {i}: Invalid data (flags, prices, or returns).")
            continue

        try:
            shares_per_dollar = 1.0 / sp
        except ZeroDivisionError:
            skipped_rows += 1
            logger.warning(f"Skipping row {i}: ZeroDivisionError calculating shares_per_dollar (sp={sp}).")
            continue

        # --- Random Agent Decision ---
        r6 = 1 if random.random() < 0.3 else 0
        r3 = 1 if random.random() < 0.2 else 0
        random_desired_invest = 0
        if r6 and r3:
            random_desired_invest = investment_both_signals
        elif r6 or r3:
            random_desired_invest = investment_per_signal

        random_invest = 0
        random_profit_this_ipo = 0
        if rnd_cap >= random_desired_invest and random_desired_invest > 0:
            random_invest = random_desired_invest
            rnd_cap -= random_invest
            shares = random_invest * shares_per_dollar
            random_sold = 0
            if random_invest == investment_per_signal and r6 == 1:
                sold_val = shares * sp6
                if pd.isna(sold_val):
                    sold_val = shares * sp
                random_sold = sold_val
            elif random_invest == investment_per_signal and r3 == 1:
                sold_val = shares * sp3
                if pd.isna(sold_val):
                    sold_val = shares * sp
                random_sold = sold_val
            elif random_invest == investment_both_signals:
                half_shares = shares / 2
                sold_6m_val = half_shares * sp6
                sold_3y_val = half_shares * sp3
                if pd.isna(sold_6m_val):
                    sold_6m_val = half_shares * sp
                if pd.isna(sold_3y_val):
                    sold_3y_val = half_shares * sp
                random_sold = sold_6m_val + sold_3y_val
            rnd_cap += random_sold
            random_profit_this_ipo = random_sold - random_invest
            # Track metrics
            if random_profit_this_ipo > 0:
                rnd_wins += 1
                rnd_win_profits.append(random_profit_this_ipo)
            elif random_profit_this_ipo < 0:
                rnd_losses += 1
                rnd_loss_profits.append(random_profit_this_ipo)
            rnd_cum += random_profit_this_ipo
            rnd_cum_profits.append(rnd_cum)
            rnd_investments.append(random_invest)

        # --- LGBM Agent Decision ---
        m6 = 1 if y_prob_6m[i] >= prediction_threshold else 0
        m3 = 1 if y_prob_3y[i] >= prediction_threshold else 0
        model_desired_invest = 0
        if m6 and m3:
            model_desired_invest = investment_both_signals
        elif m6 or m3:
            model_desired_invest = investment_per_signal

        model_invest = 0
        model_profit_this_ipo = 0
        if mdl_cap >= model_desired_invest and model_desired_invest > 0:
            model_invest = model_desired_invest
            mdl_cap -= model_invest
            shares = model_invest * shares_per_dollar
            model_sold = 0
            if model_invest == investment_per_signal and m6 == 1:
                sold_val = shares * sp6
                if pd.isna(sold_val):
                    sold_val = shares * sp
                model_sold = sold_val
            elif model_invest == investment_per_signal and m3 == 1:
                sold_val = shares * sp3
                if pd.isna(sold_val):
                    sold_val = shares * sp
                model_sold = sold_val
            elif model_invest == investment_both_signals:
                half_shares = shares / 2
                sold_6m_val = half_shares * sp6
                sold_3y_val = half_shares * sp3
                if pd.isna(sold_6m_val):
                    sold_6m_val = half_shares * sp
                if pd.isna(sold_3y_val):
                    sold_3y_val = half_shares * sp
                model_sold = sold_6m_val + sold_3y_val
            mdl_cap += model_sold
            model_profit_this_ipo = model_sold - model_invest
            # Track metrics
            if model_profit_this_ipo > 0:
                mdl_wins += 1
                mdl_win_profits.append(model_profit_this_ipo)
            elif model_profit_this_ipo < 0:
                mdl_losses += 1
                mdl_loss_profits.append(model_profit_this_ipo)
            mdl_cum += model_profit_this_ipo
            mdl_cum_profits.append(mdl_cum)
            mdl_investments.append(model_invest)
            # Track prediction accuracy
            if m6 == 1:
                mdl_predictions_6m += 1
                if int(true_6m) == 1:
                    mdl_correct_6m += 1
            if m3 == 1:
                mdl_predictions_3y += 1
                if int(true_3y) == 1:
                    mdl_correct_3y += 1

        # --- Neural Network Agent Decision ---
        n6 = 1 if y_prob_nn_6m[i] >= prediction_threshold else 0
        n3 = 1 if y_prob_nn_3y[i] >= prediction_threshold else 0
        nn_desired_invest = 0
        if n6 and n3:
            nn_desired_invest = investment_both_signals
        elif n6 or n3:
            nn_desired_invest = investment_per_signal

        nn_invest = 0
        nn_profit_this_ipo = 0
        if nn_cap >= nn_desired_invest and nn_desired_invest > 0:
            nn_invest = nn_desired_invest
            nn_cap -= nn_invest
            shares = nn_invest * shares_per_dollar
            nn_sold = 0
            if nn_invest == investment_per_signal and n6 == 1:
                sold_val = shares * sp6
                if pd.isna(sold_val):
                    sold_val = shares * sp
                nn_sold = sold_val
            elif nn_invest == investment_per_signal and n3 == 1:
                sold_val = shares * sp3
                if pd.isna(sold_val):
                    sold_val = shares * sp
                nn_sold = sold_val
            elif nn_invest == investment_both_signals:
                half_shares = shares / 2
                sold_6m_val = half_shares * sp6
                sold_3y_val = half_shares * sp3
                if pd.isna(sold_6m_val):
                    sold_6m_val = half_shares * sp
                if pd.isna(sold_3y_val):
                    sold_3y_val = half_shares * sp
                nn_sold = sold_6m_val + sold_3y_val
            nn_cap += nn_sold
            nn_profit_this_ipo = nn_sold - nn_invest
            # Track metrics
            if nn_profit_this_ipo > 0:
                nn_wins += 1
                nn_win_profits.append(nn_profit_this_ipo)
            elif nn_profit_this_ipo < 0:
                nn_losses += 1
                nn_loss_profits.append(nn_profit_this_ipo)
            nn_cum += nn_profit_this_ipo
            nn_cum_profits.append(nn_cum)
            nn_investments.append(nn_invest)
            # Track prediction accuracy
            if n6 == 1:
                nn_predictions_6m += 1
                if int(true_6m) == 1:
                    nn_correct_6m += 1
            if n3 == 1:
                nn_predictions_3y += 1
                if int(true_3y) == 1:
                    nn_correct_3y += 1

                    # Ensemble Agent (avg of LGBM & NN)
            e6 = 1 if (y_prob_6m[i] + y_prob_nn_6m[i]) / 2 >= prediction_threshold else 0
            e3 = 1 if (y_prob_3y[i] + y_prob_nn_3y[i]) / 2 >= prediction_threshold else 0

            ens_desired_invest = 0
            if e6 and e3:
                ens_desired_invest = investment_both_signals
            elif e6 or e3:
                ens_desired_invest = investment_per_signal

            ens_invest = 0
            ens_profit_this_ipo = 0
            if ens_cap >= ens_desired_invest and ens_desired_invest > 0:
                ens_invest = ens_desired_invest
                ens_cap -= ens_invest
                shares = ens_desired_invest * shares_per_dollar

                if ens_desired_invest == investment_per_signal:
                    price = sp6 if e6 else sp3
                    if pd.isna(price) or price <= 0:
                        price = sp
                    ens_sold = shares * price
                else:
                    half = shares / 2
                    v6 = half * sp6 if not pd.isna(sp6) and sp6 > 0 else half * sp
                    v3 = half * sp3 if not pd.isna(sp3) and sp3 > 0 else half * sp
                    ens_sold = v6 + v3

                ens_cap += ens_sold
                ens_profit_this_ipo = ens_sold - ens_desired_invest

                # track ensemble metrics
                if ens_profit_this_ipo > 0:
                    ens_wins += 1
                    ens_win_profits.append(ens_profit_this_ipo)
                elif ens_profit_this_ipo < 0:
                    ens_losses += 1
                    ens_loss_profits.append(ens_profit_this_ipo)
                ens_cum += ens_profit_this_ipo
                ens_cum_profits.append(ens_cum)
                ens_investments.append(ens_invest)

        # --- Save Record ---
        current_cik = df_test.loc[i, 'cik'] if 'cik' in df_test.columns else 'N/A'
        current_start_date = df_test.loc[i, 'start_date']
        history.append({
            'CIK': current_cik,
            'start_date': current_start_date.strftime('%Y-%m-%d') if pd.notna(current_start_date) else None,
            'start_price': sp if not pd.isna(sp) else np.nan,
            'six_month_price': sp6 if not pd.isna(sp6) else np.nan,
            'three_year_price': sp3 if not pd.isna(sp3) else np.nan,
            'return_6m': ret_6m if not pd.isna(ret_6m) else np.nan,
            'return_3y': ret_3y if not pd.isna(ret_3y) else np.nan,
            'random_predict': f"6M:{r6} 3Y:{r3}",
            'lgbm_predict': f"6M:{m6} 3Y:{m3}",
            'nn_predict': f"6M:{n6} 3Y:{n3}",
            'random_desired_invest': random_desired_invest,
            'random_actual_invest': random_invest,
            'random_profit_this_ipo': random_profit_this_ipo,
            'random_cumulative_profit': rnd_cum,
            'random_capital_after_ipo': rnd_cap,
            'lgbm_desired_invest': model_desired_invest,
            'lgbm_actual_invest': model_invest,
            'lgbm_profit_this_ipo': model_profit_this_ipo,
            'lgbm_cumulative_profit': mdl_cum,
            'lgbm_capital_after_ipo': mdl_cap,
            'nn_desired_invest': nn_desired_invest,
            'nn_actual_invest': nn_invest,
            'nn_profit_this_ipo': nn_profit_this_ipo,
            'nn_cumulative_profit': nn_cum,
            'nn_capital_after_ipo': nn_cap,
            'ens_desired_invest': ens_desired_invest,
            'ens_actual_invest': ens_invest,
            'ens_profit_this_ipo': ens_profit_this_ipo,
            'ens_cumulative_profit': ens_cum,
            'ens_capital_after_ipo': ens_cap,
            'lgbm_prob_6m': y_prob_6m[i],
            'lgbm_prob_3y': y_prob_3y[i],
            'nn_prob_6m': y_prob_nn_6m[i],
            'nn_prob_3y': y_prob_nn_3y[i],
            'true_6m': int(true_6m) if not pd.isna(true_6m) else np.nan,
            'true_3y': int(true_3y) if not pd.isna(true_3y) else np.nan
        })
        processed_rows += 1

        if (i + 1) % 1000 == 0:
            logger.info(f"Processed {i + 1}/{len(df_test)} simulation steps...")

except Exception as e:
    logger.error(f"Error during simulation loop at index {i}: {e}")
    logger.error(traceback.format_exc())
    raise RuntimeError("Simulation loop failed") from e

logger.info(f"Backtest simulation finished. Processed rows: {processed_rows}, Skipped rows: {skipped_rows}")

# ─── Compute Additional Metrics ───────────────────────────────
# Function to calculate maximum drawdown
def calculate_max_drawdown(cumulative_profits):
    max_drawdown = 0.0
    peak = cumulative_profits[0]
    for profit in cumulative_profits:
        if profit > peak:
            peak = profit
        drawdown = peak - profit
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

# Function to calculate simplified Sharpe Ratio
def calculate_sharpe_ratio(profits, risk_free_rate=0.0):
    if len(profits) <= 1:
        return 0.0
    returns = np.array(profits)
    avg_return = np.mean(returns)
    std_return = np.std(returns, ddof=1)
    if std_return == 0:
        return 0.0
    sharpe = (avg_return - risk_free_rate) / std_return
    return sharpe

# Random Agent Metrics
rnd_total_trades = rnd_wins + rnd_losses
rnd_win_rate = (rnd_wins / rnd_total_trades * 100) if rnd_total_trades > 0 else 0.0
rnd_avg_win = np.mean(rnd_win_profits) if rnd_win_profits else 0.0
rnd_avg_loss = np.mean(rnd_loss_profits) if rnd_loss_profits else 0.0
rnd_win_loss_ratio = rnd_wins / rnd_losses if rnd_losses > 0 else float('inf')
rnd_max_drawdown = calculate_max_drawdown(rnd_cum_profits)
rnd_avg_investment = np.mean([inv for inv in rnd_investments if inv > 0]) if any(inv > 0 for inv in rnd_investments) else 0.0
rnd_sharpe = calculate_sharpe_ratio([p for p in rnd_investments if p != 0])

# LGBM Agent Metrics
mdl_total_trades = mdl_wins + mdl_losses
mdl_win_rate = (mdl_wins / mdl_total_trades * 100) if mdl_total_trades > 0 else 0.0
mdl_avg_win = np.mean(mdl_win_profits) if mdl_win_profits else 0.0
mdl_avg_loss = np.mean(mdl_loss_profits) if mdl_loss_profits else 0.0
mdl_win_loss_ratio = mdl_wins / mdl_losses if mdl_losses > 0 else float('inf')
mdl_max_drawdown = calculate_max_drawdown(mdl_cum_profits)
mdl_avg_investment = np.mean([inv for inv in mdl_investments if inv > 0]) if any(inv > 0 for inv in mdl_investments) else 0.0
mdl_hit_rate_6m = (mdl_correct_6m / mdl_predictions_6m * 100) if mdl_predictions_6m > 0 else 0.0
mdl_hit_rate_3y = (mdl_correct_3y / mdl_predictions_3y * 100) if mdl_predictions_3y > 0 else 0.0
mdl_sharpe = calculate_sharpe_ratio([p for p in mdl_investments if p != 0])

# Neural Network Agent Metrics
nn_total_trades = nn_wins + nn_losses
nn_win_rate = (nn_wins / nn_total_trades * 100) if nn_total_trades > 0 else 0.0
nn_avg_win = np.mean(nn_win_profits) if nn_win_profits else 0.0
nn_avg_loss = np.mean(nn_loss_profits) if nn_loss_profits else 0.0
nn_win_loss_ratio = nn_wins / nn_losses if nn_losses > 0 else float('inf')
nn_max_drawdown = calculate_max_drawdown(nn_cum_profits)
nn_avg_investment = np.mean([inv for inv in nn_investments if inv > 0]) if any(inv > 0 for inv in nn_investments) else 0.0
nn_hit_rate_6m = (nn_correct_6m / nn_predictions_6m * 100) if nn_predictions_6m > 0 else 0.0
nn_hit_rate_3y = (nn_correct_3y / nn_predictions_3y * 100) if nn_predictions_3y > 0 else 0.0
nn_sharpe = calculate_sharpe_ratio([p for p in nn_investments if p != 0])

# Ensemble Agent Metrics
ens_total_trades       = ens_wins + ens_losses
ens_win_rate           = (ens_wins / ens_total_trades * 100) if ens_total_trades > 0 else 0.0
ens_avg_win            = np.mean(ens_win_profits) if ens_win_profits else 0.0
ens_avg_loss           = np.mean(ens_loss_profits) if ens_loss_profits else 0.0
ens_win_loss_ratio     = ens_wins / ens_losses if ens_losses > 0 else float('inf')
ens_max_drawdown       = calculate_max_drawdown(ens_cum_profits)
ens_avg_investment     = np.mean([inv for inv in ens_investments if inv > 0]) if any(inv > 0 for inv in ens_investments) else 0.0
ens_hit_rate_6m        = (ens_correct_6m / ens_predictions_6m * 100) if ens_predictions_6m > 0 else 0.0
ens_hit_rate_3y        = (ens_correct_3y / ens_predictions_3y * 100) if ens_predictions_3y > 0 else 0.0
ens_sharpe             = calculate_sharpe_ratio([p for p in ens_investments if p != 0])


# ─── 6) Save Results & Summary ───────────────────────────────
logger.info("Saving simulation results...")
try:
    history_df = pd.DataFrame(history)
    output_file = "ipo_investment_per_ipo_100k_chrono101.csv"
    history_df.to_csv(output_file, index=False, float_format='%.4f')
    logger.info(f"Simulation results saved to {output_file}")
except Exception as e:
    logger.error(f"Failed to save results to {output_file}: {e}")
    logger.error(traceback.format_exc())
    raise RuntimeError("Failed to save results") from e

# --- Display Summary ---
try:
    print("\n" + "=" * 10 + " Simulation Summary " + "=" * 10)
    print(f"Initial Capital per Strategy: ${initial_capital:,.2f}")
    print(f"Investment per Signal:        ${investment_per_signal:,.2f}")
    print(f"Investment for Both Signals:  ${investment_both_signals:,.2f}")
    print(f"Prediction Threshold:         {prediction_threshold:.2f}")
    print(f"Total Opportunities in Test Set: {len(df_test)}")
    print(f"Total Processed Opportunities:   {processed_rows}")
    print(f"Total Skipped Opportunities:     {skipped_rows}")
    print("-" * 55)
    print(f"{'Strategy':<18} | {'Final Capital':>15} | {'Total Profit':>15}")
    print("-" * 55)

    final_rnd_cap = initial_capital + rnd_cum
    final_mdl_cap = initial_capital + mdl_cum
    final_nn_cap = initial_capital + nn_cum

    print(f"{'Random':<18} | ${final_rnd_cap:14,.2f} | ${rnd_cum:14,.2f}")
    print(f"{'LGBM':<18} | ${final_mdl_cap:14,.2f} | ${mdl_cum:14,.2f}")
    print(f"{'Neural Network':<18} | ${final_nn_cap:14,.2f} | ${nn_cum:14,.2f}")
    print(f"{'Ensemble':<18} | ${initial_capital + ens_cum:14,.2f} | ${ens_cum:14,.2f}")
    print("-" * 55)

    # --- Extended Metrics Summary ---
    print("\n" + "=" * 10 + " Extended Metrics " + "=" * 10)
    print(f"{'Metric':<25} | {'Random':>10} | {'LGBM':>10} | {'Neural Network':>15} |{'Ensemble':>15}")
    print("-" * 65)

    print(f"{'Win Rate (%)':<25} | {rnd_win_rate:>10.2f} | {mdl_win_rate:>10.2f} | {nn_win_rate:>15.2f} | {ens_win_rate:>10.2f}")
    print(f"{'Total Trades':<25} | {rnd_total_trades:>10} | {mdl_total_trades:>10} | {nn_total_trades:>15} | {ens_total_trades:>10}")
    print(f"{'Average Win ($)':<25} | {rnd_avg_win:>10.2f} | {mdl_avg_win:>10.2f} | {nn_avg_win:>15.2f} | {ens_avg_win:>10.2f}")
    print(f"{'Average Loss ($)':<25} | {rnd_avg_loss:>10.2f} | {mdl_avg_loss:>10.2f} | {nn_avg_loss:>15.2f} | {ens_avg_loss:>10.2f}")
    print(f"{'Win/Loss Ratio':<25} | {rnd_win_loss_ratio:>10.2f} | {mdl_win_loss_ratio:>10.2f} | {nn_win_loss_ratio:>15.2f} | {ens_win_loss_ratio:>10.2f}")
    print(f"{'Max Drawdown ($)':<25} | {rnd_max_drawdown:>10.2f} | {mdl_max_drawdown:>10.2f} | {nn_max_drawdown:>15.2f} | {ens_max_drawdown:>10.2f}")
    print(f"{'Avg Investment ($)':<25} | {rnd_avg_investment:>10.2f} | {mdl_avg_investment:>10.2f} | {nn_avg_investment:>15.2f} | {ens_avg_investment:>10.2f}")
    print(f"{'Sharpe Ratio':<25} | {rnd_sharpe:>10.2f} | {mdl_sharpe:>10.2f} | {nn_sharpe:>15.2f} | {ens_sharpe:>10.2f}")
    #print(f"{'Hit Rate 6M (%)':<25} | {'N/A':>10} | {mdl_hit_rate_6m:>10.2f} | {nn_hit_rate_6m:>15.2f} | {ens_hit_rate_6m:>10.2f}")
    #print(f"{'Hit Rate 3Y (%)':<25} | {'N/A':>10} | {mdl_hit_rate_3y:>10.2f} | {nn_hit_rate_3y:>15.2f} | {ens_hit_rate_3y:>10.2f}")
    print("-" * 80)


except Exception as e:
    logger.error(f"Error displaying summary: {e}")
    logger.error(traceback.format_exc())
    raise RuntimeError("Failed to display summary") from e

logger.info("Script finished.")

In [ ]:
#SHAP values

import os
import json
import joblib
import numpy as np
import pandas as pd
import shap
from lightgbm import Booster
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# ── CONFIG ─────────────────────────────────────────────────────────────────────
OUTPUT_DIR = '/scratch/ghtillem/Saved_LGBM_Models/Final/final'
DATA_FILE   = '/scratch/ghtillem/Data/EODHD_Final_maxcik50_flat_wsf_incorp_m.csv'
TARGETS     = ['flag_3y_avg', 'flag_6m_avg']
LEAKAGE = [
    'six_month_price', 'three_year_price', 'six_month_30d_avg', 'three_year_30d_avg',
    'flag_6m_price', 'flag_3y_price', 'six_month_date', 'three_year_date',
    'sp500_sma30_centered_at_start', 'sp500_above_30sma_centered', '_merge',
    'cik', 'title', '1Y_zipma_ROC', 'All_zipba_ROC', '1Y_zipba_ROC', 'All_sic_ROC',
    '1Y_fye_ROC', '1Y_sic_ROC', '3Y_zipma_ROC', '3Y_zipba_ROC', 'All_median_zipba',
    'Q1_zipba_median', 'Q1_sic_mean_y', 'Q4_sic_median_y', 'Q1_zipma_median_y',
    'Q2_sic_mean_x', 'Q2_sic_median', 'Q0_sic_mean', 'Q0_zipba_mean', 'sic',
    'Q1_zipma_median_x', 'Q1_zippa_mean_x', 'Q1_fye_median_x', 'Q0_fye_mean',
    'Q1_zipba_median_y', 'Q0_zipma_ROC_x', 'Q4_sic_mean_y', 'Q0_zipba_mean_y',
    'Q0_zipba_mean_x', 'Q4_zipma_median_x', 'Q2_fye_mean_x', 'bus_zip', 'Q4_fye_median',
    'Q2_fye_mean_y', 'Q1_fye_mean', 'Q0_zipma_median_y', 'Q1_fye_median_y', 'Q0_fye_median_y',
    'phone', 'ticker', 'start_date'
]  # same list you used in training
TEST_SIZE   = 0.30
RANDOM_SEED = 42
SAMPLE_N    = None

# ── LOAD DATA ─────────────────────────────────────────────────────────────────
df = pd.read_csv(DATA_FILE, low_memory=False)
df.drop(columns=[c for c in LEAKAGE if c not in TARGETS], errors='ignore', inplace=True)

# ── MAIN LOOP ─────────────────────────────────────────────────────────────────
for target in TARGETS:
    print(f"\n===== SHAP for {target} =====")
    # --- reload preprocessor & metadata ---
    proc_candidates = sorted(f for f in os.listdir(OUTPUT_DIR)
                             if f.startswith(f"{target}_preprocessor_") and f.endswith('.joblib'))
    preprocessor: Pipeline = joblib.load(os.path.join(OUTPUT_DIR, proc_candidates[-1]))
    meta_candidates = sorted(f for f in os.listdir(OUTPUT_DIR)
                             if f.startswith(f"{target}_preprocessor_metadata_") and f.endswith('.json'))
    metadata = json.load(open(os.path.join(OUTPUT_DIR, meta_candidates[-1])))
    feature_names = metadata['feature_names']

    # --- reload LightGBM booster ---
    model_txts = sorted(f for f in os.listdir(OUTPUT_DIR)
                        if f.startswith(target) and f.endswith('.txt'))
    booster = Booster(model_file=os.path.join(OUTPUT_DIR, model_txts[-1]))

    # --- rebuild test split ---
    y = df[target].dropna().astype(int)
    X = df.loc[y.index].drop(columns=[target])
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_SEED
    )

    # --- optionally subsample ---
    if SAMPLE_N is not None and SAMPLE_N < len(X_test):
        X_test_small = X_test.sample(n=SAMPLE_N, random_state=RANDOM_SEED)
    else:
        X_test_small = X_test

    # --- preprocess ---
    X_pre = preprocessor.transform(X_test_small)

    # --- SHAP explainer on probability scale ---
    explainer = shap.TreeExplainer(
        booster,
        model_output="probability",
        data=X_pre  # speeds up init
    )
    raw_shap = explainer.shap_values(X_pre)
    # binary classifier: pick the positive‐class matrix
    shap_vals = raw_shap[1] if isinstance(raw_shap, list) else raw_shap

    # --- sanity check ---
    assert shap_vals.shape == X_pre.shape, f"SHAP shape mismatch for {target}"

    # --- aggregate importances ---
    mean_abs = np.abs(shap_vals).mean(axis=0)
    imp_df = (pd.DataFrame({
                  'feature': feature_names,
                  'mean_abs_shap': mean_abs
              })
              .sort_values('mean_abs_shap', ascending=False))

    # --- report & save ---
    print(f"\nTop 20 features for {target} (n={len(X_test_small)} test rows):")
    print(imp_df.head(20).to_string(index=False))
    out_csv = os.path.join(OUTPUT_DIR, f"{target}_shap_importance_prob_fulltest.csv")
    imp_df.to_csv(out_csv, index=False)
    print(f"→ Saved full ranking to {out_csv}")


In [ ]:
#Now we filter the SEC Filings data by the CIK to tickers SEC data we optained (using the unique CIK file)

import pandas as pd

# ✅ File paths
input_file = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/SEC_filings_cleaned_final10.csv"
output_file = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/ipo_filings_filtered_by_cik.csv"
ipo_cik_file = "/content/drive/MyDrive/FSE IPO Data/Filtered Data/Final/cik_unique.csv"

# ✅ Load distinct CIKs
ipo_ciks = pd.read_csv(ipo_cik_file)
ipo_ciks['cik'] = ipo_ciks['cik'].astype(str).str.zfill(10)
ipo_cik_set = set(ipo_ciks['cik'].unique())

# ✅ Chunk processing setup
chunksize = 500000
first = True

# ✅ Process input file in chunks
for chunk in pd.read_csv(input_file, chunksize=chunksize):
    # Normalize cik format
    chunk['cik'] = chunk['cik'].apply(lambda x: str(int(x)).zfill(10))

    # Filter only rows where cik is in our IPO list
    filtered = chunk[chunk['cik'].isin(ipo_cik_set)]

    # Write out filtered rows
    filtered.to_csv(output_file, mode='a', index=False, header=first)
    first = False

    print(f"✅ Processed {len(chunk):,} rows; retained {len(filtered):,} matching CIKs")